In [ ]:
#pip install statsmodels
#!pip install matplotlib
#!pip install meteostat


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Import Meteostat library and dependencies
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily
import pandas as pd
import glob

In [10]:
import os

# Print the current working directory
print(os.getcwd())

c:\Users\dhihr\OneDrive\bahan who\kerjaan who\climate


In [11]:
os.chdir('C:\\Users\\dhihr\\OneDrive\\bahan who\\kerjaan who\\climate')

# Find all .csv files
csv_files = glob.glob("*.csv")

# Delete each file
for file in csv_files:
    try:
        os.remove(file)
        print(f"Deleted: {file}")
    except Exception as e:
        print(f"Error deleting {file}: {e}")


Deleted: bangladesh_country_weather_with_prediction_2022_2025.csv
Deleted: bangladesh_states_weather_daily_2022_2025.csv
Deleted: bangladesh_states_weather_with_prediction_2022_2025.csv
Deleted: bangladesh_weekly_states_weather_daily_2022_2025.csv
Deleted: bhutan_country_weather_with_prediction_2022_2025.csv
Deleted: bhutan_states_weather_daily_2022_2025.csv
Deleted: bhutan_states_weather_with_prediction_2022_2025.csv
Deleted: combined_country_weather_with_prediction_2022_2025.csv
Deleted: combined_states_weather_with_prediction_2022_2025.csv
Deleted: india_country_weather_with_prediction_2022_2025.csv
Deleted: india_states_weather_daily_2022_2025.csv
Deleted: india_states_weather_with_prediction_2022_2025.csv
Deleted: maldives_country_weather_with_prediction_2022_2025.csv
Deleted: maldives_states_weather_daily_2022_2025.csv
Deleted: maldives_states_weather_with_prediction_2022_2025.csv
Deleted: myanmar_country_weather_with_prediction_2022_2025.csv
Deleted: myanmar_states_weather_daily

## Bangladesh

### Daily Historical Data

In [5]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import numpy as np

# Define time period
start = datetime(2017, 1, 1)
end = datetime(2025, 12, 31, 23, 59)

# Define major administrative divisions or cities in Bangladesh
bangladesh_divisions = {
    'Dhaka': Point(23.8103, 90.4125, 4),
    'Chattogram': Point(22.3569, 91.7832, 9),
    'Khulna': Point(22.8456, 89.5403, 8),
    'Rajshahi': Point(24.3745, 88.6042, 18),
    'Sylhet': Point(24.8949, 91.8687, 35),
    'Barisal': Point(22.7010, 90.3535, 3),
    'Rangpur': Point(25.7480, 89.2390, 34),
    'Mymensingh': Point(24.7471, 90.4203, 19)
}

# Initialize dictionary to store DataFrames
all_data = {}

# ✅ Function to calculate Relative Humidity using Magnus formula
def calc_rh(temp, dewpt):
    temp = np.array(temp, dtype=float)
    dewpt = np.array(dewpt, dtype=float)
    mask = (~np.isnan(temp)) & (~np.isnan(dewpt))
    rh = np.full(temp.shape, np.nan)
    rh[mask] = 100 * np.exp((17.625 * dewpt[mask]) / (243.04 + dewpt[mask])) / \
                      np.exp((17.625 * temp[mask]) / (243.04 + temp[mask]))
    return rh

# ✅ Fetch hourly data for each division
for division, point in bangladesh_divisions.items():
    try:
        print(f"Fetching hourly data for {division}...")
        data = Hourly(point, start, end).fetch()

        # Skip if no data
        if data.empty:
            print(f"No data available for {division}")
            continue

        data['Division'] = division

        # Calculate RH if missing
        if 'rhum' not in data.columns and 'temp' in data.columns and 'dwpt' in data.columns:
            data['rhum'] = calc_rh(data['temp'].values, data['dwpt'].values)

        all_data[division] = data

    except Exception as e:
        print(f"Failed to fetch data for {division}: {e}")

# ✅ Combine all data into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data.values()).reset_index()
else:
    combined_df = pd.DataFrame()
    print("No data fetched for any division.")

# ✅ Ensure 'time' is datetime
if not combined_df.empty:
    combined_df['time'] = pd.to_datetime(combined_df['time'])

    # ✅ Group by Division and resample to daily
    daily_df = combined_df.groupby('Division').resample('D', on='time').agg({
        'temp': ['mean', 'max', 'min'],   # Avg, Max, Min temperature
        'rhum': 'mean',                   # Avg relative humidity
        'prcp': 'sum',                    # Total daily precipitation
        'wspd': 'mean',                   # Avg wind speed
        'tsun': 'sum'                     # Sum of sun exposure
    }).reset_index()

    # ✅ Flatten multi-level columns
    daily_df.columns = ['Division', 'time', 'temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    # ✅ Add 'Country' column
    daily_df['Country'] = 'Bangladesh'

     # ✅ Create provisional columns (copy today & future values)
    today = datetime.now().date()
    climate_vars = ['temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

for col in climate_vars:
    new_col = f"{col}_provisional"
    # ✅ Copy today's and future values into _provisional
    daily_df[new_col] = np.where(daily_df['time'].dt.date >= today, daily_df[col], np.nan)

    # ✅ Clear only future values from original columns, keep today intact
    daily_df.loc[daily_df['time'].dt.date > today, col] = np.nan

    # ✅ Round numeric columns to 2 decimal places
    daily_df = daily_df.round(2)

    # ✅ Save to CSV
    daily_df.to_csv("bangladesh_states_weather_daily_2022_2025.csv", index=False)

    # ✅ Check last few rows
    print(daily_df.tail())
else:
    print("Daily DataFrame is empty.")


Fetching hourly data for Dhaka...


Fetching hourly data for Chattogram...
Fetching hourly data for Khulna...


Fetching hourly data for Rajshahi...
Fetching hourly data for Sylhet...
Fetching hourly data for Barisal...


Fetching hourly data for Rangpur...


Fetching hourly data for Mymensingh...
Failed to fetch data for Mymensingh: 'Hourly' object has no attribute '_types'
      Division       time  temp_mean  temp_max  temp_min  rhum_mean  prcp_sum  \
20217   Sylhet 2025-10-10       <NA>      29.0      23.3       88.0       0.0   
20218   Sylhet 2025-10-11       <NA>      27.3      23.4      92.25       0.0   
20219   Sylhet 2025-10-12       <NA>      28.0      23.3      89.25       0.0   
20220   Sylhet 2025-10-13       <NA>      29.6      23.5      85.25       0.0   
20221   Sylhet 2025-10-14       <NA>      30.9      22.1       82.0       0.0   

       wspd_mean  tsun_sum     Country  temp_mean_provisional  
20217       3.52       0.0  Bangladesh                  25.98  
20218       3.88       0.0  Bangladesh                  25.02  
20219       3.62       0.0  Bangladesh                  25.45  
20220       1.78       0.0  Bangladesh                  26.05  
20221        3.0       0.0  Bangladesh                  26.67  
      Divis

### Weekly Data

In [6]:
import pandas as pd
import numpy as np

# Ensure time column is datetime
daily_df["time"] = pd.to_datetime(daily_df["time"])

# Coerce numeric columns (avoid agg errors if any are strings)
num_cols = ["temp_mean","temp_max","temp_min","rhum_mean","prcp_sum","wspd_mean","tsun_sum"]
for c in num_cols:
    daily_df[c] = pd.to_numeric(daily_df[c], errors="coerce")

# Set as index for resampling
df = daily_df.set_index("time")

# Group by Division & Country, then resample weekly (weeks end on Sunday)
weekly_data = (
    df.groupby(["Division", "Country"])
      .resample("W")
      .agg({
          "temp_mean": "mean",
          "temp_max": "mean",
          "temp_min": "mean",
          "rhum_mean": "mean",
          "prcp_sum": lambda s: s.sum(min_count=1),
          "wspd_mean": "mean",
          "tsun_sum": lambda s: s.sum(min_count=1)
      })
      .reset_index()
)

weekly_data[weekly_data.select_dtypes(include="number").columns] = \
    weekly_data.select_dtypes(include="number").round(2)


print(weekly_data.tail())

# Save (no leading space!)
weekly_data.to_csv("bangladesh_weekly_states_weather_daily_2022_2025.csv", index=False)



     Division     Country       time  temp_mean  temp_max  temp_min  \
2894   Sylhet  Bangladesh 2025-09-21      27.06     30.87      24.4   
2895   Sylhet  Bangladesh 2025-09-28      28.61     33.11     24.84   
2896   Sylhet  Bangladesh 2025-10-05      26.54      30.7     23.89   
2897   Sylhet  Bangladesh 2025-10-12       <NA>      <NA>      <NA>   
2898   Sylhet  Bangladesh 2025-10-19       <NA>      <NA>      <NA>   

      rhum_mean  prcp_sum  wspd_mean  tsun_sum  
2894      89.63      82.3       5.94       0.0  
2895      84.61      60.8       7.22       0.0  
2896      88.59     121.7       6.47       0.0  
2897       <NA>      <NA>       <NA>      <NA>  
2898       <NA>      <NA>       <NA>      <NA>  


### Forecast

In [7]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

# ✅ Ensure 'time' is datetime
daily_df['time'] = pd.to_datetime(daily_df['time'])

# ✅ Remove provisional data before forecasting
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]

# ✅ Drop week_start only if exists
if 'week_start' in daily_df.columns:
    daily_df = daily_df.drop(columns=['week_start'])

forecast_list = []

# ✅ Loop through each Division and forecast all variables
for division in daily_df['Division'].unique():
    division_data = daily_df[daily_df['Division'] == division].copy()
    division_data.set_index('time', inplace=True)

    if division_data.empty:
        continue

    # Get the last confirmed date for this division
    last_date = division_data.index.max()

    # Temporary dict to store forecasts for this division
    division_forecasts = {'Division': [], 'time': []}

    for var in climate_vars:
        series = division_data[var].dropna()

        if len(series) > 30:
            try:
                # Fit SARIMAX model
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                # ✅ Forecast next 7 days
                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                # Initialize division & dates on first variable
                if var == climate_vars[0]:
                    division_forecasts['Division'] = [division] * 7
                    division_forecasts['time'] = future_dates

                # Add forecast and CI columns
                division_forecasts[var] = forecast.values
                division_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                division_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {division}: {e}")
                division_forecasts[var] = [None] * 7
                division_forecasts[f"{var}_lower95"] = [None] * 7
                division_forecasts[f"{var}_upper95"] = [None] * 7
        else:
            # If not enough data, fill with None
            division_forecasts[var] = [None] * 7
            division_forecasts[f"{var}_lower95"] = [None] * 7
            division_forecasts[f"{var}_upper95"] = [None] * 7

    # Create DataFrame for this division forecast
    forecast_df = pd.DataFrame(division_forecasts)
    forecast_df['Country'] = 'Bangladesh'
    forecast_list.append(forecast_df)

# ✅ Combine forecasts for all divisions
if forecast_list:
    forecast_df_all = pd.concat(forecast_list, ignore_index=True)

    # ✅ Rename columns to avoid overwriting historical values
    for col in climate_vars:
        forecast_df_all.rename(columns={
            col: f"{col}_forecast",
            f"{col}_lower95": f"{col}_forecast_lower95",
            f"{col}_upper95": f"{col}_forecast_upper95"
        }, inplace=True)

    # ✅ Merge forecasts into historical daily_df
    combined_df_with_forecast = pd.merge(
        daily_df,
        forecast_df_all,
        on=['Division', 'time', 'Country'],
        how='outer'
    )

    # ✅ Ensure datetime for merged DataFrame
    combined_df_with_forecast['time'] = pd.to_datetime(combined_df_with_forecast['time'])

    # ✅ Copy today's actual values into _forecast columns
    for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

 # ✅ Remove all *_provisional columns
    combined_df_with_forecast = combined_df_with_forecast.drop(
        columns=[col for col in combined_df_with_forecast.columns if col.endswith('_provisional')],
        errors='ignore'
    )

    # ✅ Sort and round
    combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Division', 'time']).reset_index(drop=True)
    combined_df_with_forecast = combined_df_with_forecast.round(2)

    # ✅ Save to CSV
    combined_df_with_forecast.to_csv("bangladesh_states_weather_with_prediction_2022_2025.csv", index=False)

    # ✅ Preview last rows
    print(combined_df_with_forecast.tail(10))
else:
    print("No forecasts generated. Check if data is sufficient.")


      Division       time  temp_mean  temp_max  temp_min  rhum_mean  prcp_sum  \
20194   Sylhet 2025-10-03      26.18      30.4      23.7      90.08      19.0   
20195   Sylhet 2025-10-04       26.0      30.8      23.6      90.58      26.2   
20196   Sylhet 2025-10-05      25.65      30.3      23.4      91.17      25.6   
20197   Sylhet 2025-10-06       <NA>      <NA>      <NA>       <NA>      <NA>   
20198   Sylhet 2025-10-07       <NA>      <NA>      <NA>       <NA>      <NA>   
20199   Sylhet 2025-10-08       <NA>      <NA>      <NA>       <NA>      <NA>   
20200   Sylhet 2025-10-09       <NA>      <NA>      <NA>       <NA>      <NA>   
20201   Sylhet 2025-10-10       <NA>      <NA>      <NA>       <NA>      <NA>   
20202   Sylhet 2025-10-11       <NA>      <NA>      <NA>       <NA>      <NA>   
20203   Sylhet 2025-10-12       <NA>      <NA>      <NA>       <NA>      <NA>   

       wspd_mean  tsun_sum     Country  ...  rhum_mean_forecast_upper95  \
20194       6.33       0.0  Bangl

### Forecast National Level

In [8]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]
daily_df = daily_df.drop(columns=['week_start'], errors='ignore')

# ✅ Aggregate to Country-level mean per day
country_df = (daily_df
              .groupby(['Country', 'time'])[climate_vars]
              .mean()
              .reset_index())

forecast_list = []

# ✅ Loop through each Country and forecast all variables
for country in country_df['Country'].unique():
    country_data = country_df[country_df['Country'] == country].copy()
    country_data.set_index('time', inplace=True)

    last_date = country_data.index.max()
    country_forecasts = {'Country': [], 'time': []}

    for var in climate_vars:
        series = country_data[var].dropna()

        if len(series) > 30:
            try:
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                if var == climate_vars[0]:
                    country_forecasts['Country'] = [country] * 7
                    country_forecasts['time'] = future_dates

                country_forecasts[var] = forecast.values
                country_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                country_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {country}: {e}")
                country_forecasts[var] = [None] * 7
                country_forecasts[f"{var}_lower95"] = [None] * 7
                country_forecasts[f"{var}_upper95"] = [None] * 7

    forecast_df = pd.DataFrame(country_forecasts)
    forecast_list.append(forecast_df)

# ✅ Combine all country forecasts
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename forecast columns
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical aggregated data
combined_df_with_forecast = pd.merge(
    country_df,
    forecast_df_all,
    on=['Country', 'time'],
    how='outer'
)

# ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Country', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("bangladesh_country_weather_with_prediction_2022_2025.csv", index=False)

# Preview
print(combined_df_with_forecast.tail(10))



         Country       time  temp_mean  rhum_mean  prcp_sum  wspd_mean  \
3197  Bangladesh 2025-10-03       26.8      89.96      9.83       7.02   
3198  Bangladesh 2025-10-04      27.33      88.55     13.17       6.82   
3199  Bangladesh 2025-10-05      26.95      89.19      13.8       6.12   
3200  Bangladesh 2025-10-06       <NA>       <NA>      <NA>       <NA>   
3201  Bangladesh 2025-10-07       <NA>       <NA>      <NA>       <NA>   
3202  Bangladesh 2025-10-08       <NA>       <NA>      <NA>       <NA>   
3203  Bangladesh 2025-10-09       <NA>       <NA>      <NA>       <NA>   
3204  Bangladesh 2025-10-10       <NA>       <NA>      <NA>       <NA>   
3205  Bangladesh 2025-10-11       <NA>       <NA>      <NA>       <NA>   
3206  Bangladesh 2025-10-12       <NA>       <NA>      <NA>       <NA>   

      tsun_sum  temp_mean_forecast  temp_mean_forecast_lower95  \
3197       0.0                 NaN                         NaN   
3198       0.0                 NaN                   

## India

### Daily Hisotrical Data

In [9]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import numpy as np

# Define time period
start = datetime(2017, 1, 1)
end = datetime(2025, 12, 31, 23, 59)

# Define major administrative divisions or cities in Bangladesh
# Define Indian state capitals with approximate (lat, lon, elevation in meters)
indian_states = {
    'Andhra Pradesh': Point(16.5062, 80.6480, 30),     # Amaravati/Vijayawada
    'Arunachal Pradesh': Point(27.0844, 93.6053, 1627), # Itanagar
    'Assam': Point(26.1445, 91.7362, 55),               # Dispur (Guwahati)
    'Bihar': Point(25.5941, 85.1376, 53),               # Patna
    'Chhattisgarh': Point(21.2514, 81.6296, 298),       # Raipur
    'Goa': Point(15.2993, 74.1240, 7),                  # Panaji
    'Gujarat': Point(23.0225, 72.5714, 53),             # Gandhinagar/Ahmedabad
    'Haryana': Point(29.0588, 76.0856, 220),            # Chandigarh
    'Himachal Pradesh': Point(31.1048, 77.1734, 2200),  # Shimla
    'Jharkhand': Point(23.6102, 85.2799, 200),          # Ranchi
    'Karnataka': Point(12.9716, 77.5946, 920),          # Bengaluru
    'Kerala': Point(8.5241, 76.9366, 4),                # Thiruvananthapuram
    'Madhya Pradesh': Point(23.2599, 77.4126, 500),     # Bhopal
    'Maharashtra': Point(18.5204, 73.8567, 560),        # Pune/Mumbai
    'Manipur': Point(24.8170, 93.9368, 790),            # Imphal
    'Meghalaya': Point(25.5788, 91.8933, 1496),         # Shillong
    'Mizoram': Point(23.7271, 92.7176, 1132),           # Aizawl
    'Nagaland': Point(25.6751, 94.1086, 1444),          # Kohima
    'Odisha': Point(20.2961, 85.8245, 45),              # Bhubaneswar
    'Punjab': Point(30.7333, 76.7794, 310),             # Chandigarh
    'Rajasthan': Point(26.9124, 75.7873, 431),          # Jaipur
    'Sikkim': Point(27.3389, 88.6065, 1650),            # Gangtok
    'Tamil Nadu': Point(13.0827, 80.2707, 6),           # Chennai
    'Telangana': Point(17.3850, 78.4867, 500),          # Hyderabad
    'Tripura': Point(23.8315, 91.2868, 13),             # Agartala
    'Uttar Pradesh': Point(26.8467, 80.9462, 123),      # Lucknow
    'Uttarakhand': Point(30.3165, 78.0322, 690),        # Dehradun
    'West Bengal': Point(22.5726, 88.3639, 9),          # Kolkata
    'Delhi': Point(28.6139, 77.2090, 216),              # New Delhi

    # --- Union Territories ---
    'Andaman and Nicobar Islands': Point(11.6234, 92.7265, 16),   # Port Blair
    'Chandigarh': Point(30.7333, 76.7794, 321),                   # Chandigarh (separate UT)
    'Dadra and Nagar Haveli and Daman and Diu': Point(20.3974, 72.8328, 5), # Daman
    'Jammu & Kashmir': Point(34.0837, 74.7973, 1585),             # Srinagar
    'Ladakh': Point(34.1526, 77.5770, 3500),                      # Leh
    'Lakshadweep': Point(10.5667, 72.6417, 2),                    # Kavaratti
    'Puducherry': Point(11.9416, 79.8083, 15)                     # Puducherry
}


# Initialize dictionary to store DataFrames
all_data = {}

# ✅ Function to calculate Relative Humidity using Magnus formula
def calc_rh(temp, dewpt):
    temp = np.array(temp, dtype=float)
    dewpt = np.array(dewpt, dtype=float)
    mask = (~np.isnan(temp)) & (~np.isnan(dewpt))
    rh = np.full(temp.shape, np.nan)
    rh[mask] = 100 * np.exp((17.625 * dewpt[mask]) / (243.04 + dewpt[mask])) / \
                      np.exp((17.625 * temp[mask]) / (243.04 + temp[mask]))
    return rh

# ✅ Fetch hourly data for each division
for division, point in indian_states.items():
    try:
        print(f"Fetching hourly data for {division}...")
        data = Hourly(point, start, end).fetch()

        # Skip if no data
        if data.empty:
            print(f"No data available for {division}")
            continue

        data['Division'] = division

        # Calculate RH if missing
        if 'rhum' not in data.columns and 'temp' in data.columns and 'dwpt' in data.columns:
            data['rhum'] = calc_rh(data['temp'].values, data['dwpt'].values)

        all_data[division] = data

    except Exception as e:
        print(f"Failed to fetch data for {division}: {e}")

# ✅ Combine all data into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data.values()).reset_index()
else:
    combined_df = pd.DataFrame()
    print("No data fetched for any division.")

# ✅ Ensure 'time' is datetime
if not combined_df.empty:
    combined_df['time'] = pd.to_datetime(combined_df['time'])

    # ✅ Group by Division and resample to daily
    daily_df = combined_df.groupby('Division').resample('D', on='time').agg({
        'temp': ['mean', 'max', 'min'],   # Avg, Max, Min temperature
        'rhum': 'mean',                   # Avg relative humidity
        'prcp': 'sum',                    # Total daily precipitation
        'wspd': 'mean',                   # Avg wind speed
        'tsun': 'sum'                     # Sum of sun exposure
    }).reset_index()

    # ✅ Flatten multi-level columns
    daily_df.columns = ['Division', 'time', 'temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    # ✅ Add 'Country' column
    daily_df['Country'] = 'India'

     # ✅ Create provisional columns (copy today & future values)
    today = datetime.now().date()
    climate_vars = ['temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    for col in climate_vars:
        new_col = f"{col}_provisional"
        # Copy today's and future values into _provisional
        daily_df[new_col] = np.where(daily_df['time'].dt.date >= today, daily_df[col], np.nan)

        # Clear only future values from original columns, keep today intact
        daily_df.loc[daily_df['time'].dt.date > today, col] = np.nan

    # ✅ Round and save
    daily_df = daily_df.round(2)

    # ✅ Save to CSV
    daily_df.to_csv("india_states_weather_daily_2022_2025.csv", index=False)

    # ✅ Check last few rows
    print(daily_df.tail())
else:
    print("Daily DataFrame is empty.")

Fetching hourly data for Andhra Pradesh...
Fetching hourly data for Arunachal Pradesh...
Failed to fetch data for Arunachal Pradesh: 'Hourly' object has no attribute '_types'
Fetching hourly data for Assam...
Fetching hourly data for Bihar...
Fetching hourly data for Chhattisgarh...


Fetching hourly data for Goa...
Fetching hourly data for Gujarat...


Fetching hourly data for Haryana...
Failed to fetch data for Haryana: 'Hourly' object has no attribute '_types'
Fetching hourly data for Himachal Pradesh...


Fetching hourly data for Jharkhand...
Failed to fetch data for Jharkhand: 'Hourly' object has no attribute '_types'
Fetching hourly data for Karnataka...
Fetching hourly data for Kerala...


Fetching hourly data for Madhya Pradesh...
Fetching hourly data for Maharashtra...


...

Fetching hourly data for Uttarakhand...


Fetching hourly data for West Bengal...
Fetching hourly data for Delhi...
Fetching hourly data for Andaman and Nicobar Islands...


Fetching hourly data for Chandigarh...
Fetching hourly data for Dadra and Nagar Haveli and Daman and Diu...


Fetching hourly data for Jammu & Kashmir...
Fetching hourly data for Ladakh...
Failed to fetch data for Ladakh: 'Hourly' object has no attribute '_types'
Fetching hourly data for Lakshadweep...
Failed to fetch data for Lakshadweep: 'Hourly' object has no attribute '_types'
Fetching hourly data for Puducherry...


          Division       time  temp_mean  temp_max  temp_min  rhum_mean  \
96275  West Bengal 2025-10-11       <NA>      <NA>      <NA>       <NA>   
96276  West Bengal 2025-10-12       <NA>      <NA>      <NA>       <NA>   
96277  West Bengal 2025-10-13       <NA>      <NA>      <NA>       <NA>   
96278  West Bengal 2025-10-14       <NA>      <NA>      <NA>       <NA>   
96279  West Bengal 2025-10-15       <NA>      <NA>      <NA>       <NA>   

       prcp_sum  wspd_mean  tsun_sum Country  temp_mean_provisional  \
96275      <NA>       <NA>      <NA>   India                  28.28   
96276      <NA>       <NA>      <NA>   India                  28.23   
96277      <NA>       <NA>      <NA>   India                  27.85   
96278      <NA>       <NA>      <NA>   India                  27.79   
96279      <NA>       <NA>      <NA>   India                  28.99   

       temp_max_provisional  temp_min_provisional  rhum_mean_provisional  \
96275                  32.1                  2

### Forecast

In [10]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data before forecasting
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]


forecast_list = []

# ✅ Loop through each Division and forecast all variables
for division in daily_df['Division'].unique():
    division_data = daily_df[daily_df['Division'] == division].copy()
    division_data.set_index('time', inplace=True)

    # Get the last confirmed date for this division
    last_date = division_data.index.max()

    # Temporary dict to store forecasts for this division
    division_forecasts = {'Division': [], 'time': []}

    for var in climate_vars:
        series = division_data[var].dropna()

        if len(series) > 30:
            try:
                # Fit SARIMAX model
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                # ✅ Forecast next 7 days
                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                # Initialize division & dates on first variable
                if var == climate_vars[0]:
                    division_forecasts['Division'] = [division] * 7
                    division_forecasts['time'] = future_dates

                # Add forecast and CI columns
                division_forecasts[var] = forecast.values
                division_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                division_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {division}: {e}")
                division_forecasts[var] = [None] * 7
                division_forecasts[f"{var}_lower95"] = [None] * 7
                division_forecasts[f"{var}_upper95"] = [None] * 7

    # Create DataFrame for this division forecast
    forecast_df = pd.DataFrame(division_forecasts)
    forecast_df['Country'] = 'India'
    forecast_list.append(forecast_df)

# ✅ Combine forecasts for all divisions
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename columns to avoid overwriting historical values
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical daily_df
combined_df_with_forecast = pd.merge(
    daily_df,
    forecast_df_all,
    on=['Division', 'time', 'Country'],
    how='outer'
)

# ✅ Ensure datetime for merged DataFrame
combined_df_with_forecast['time'] = pd.to_datetime(combined_df_with_forecast['time'])

    # ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

 # ✅ Remove all *_provisional columns
combined_df_with_forecast = combined_df_with_forecast.drop(
        columns=[col for col in combined_df_with_forecast.columns if col.endswith('_provisional')],
        errors='ignore'
    )


# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Division', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("india_states_weather_with_prediction_2022_2025.csv", index=False)

# Preview last rows
print(combined_df_with_forecast.tail(10))

          Division       time  temp_mean  temp_max  temp_min  rhum_mean  \
96196  West Bengal 2025-10-03      26.65      28.9      24.5      89.04   
96197  West Bengal 2025-10-04      26.56      29.4      24.7      89.96   
96198  West Bengal 2025-10-05      26.18      29.5      24.1      91.33   
96199  West Bengal 2025-10-06       <NA>      <NA>      <NA>       <NA>   
96200  West Bengal 2025-10-07       <NA>      <NA>      <NA>       <NA>   
96201  West Bengal 2025-10-08       <NA>      <NA>      <NA>       <NA>   
96202  West Bengal 2025-10-09       <NA>      <NA>      <NA>       <NA>   
96203  West Bengal 2025-10-10       <NA>      <NA>      <NA>       <NA>   
96204  West Bengal 2025-10-11       <NA>      <NA>      <NA>       <NA>   
96205  West Bengal 2025-10-12       <NA>      <NA>      <NA>       <NA>   

       prcp_sum  wspd_mean  tsun_sum Country  ...  rhum_mean_forecast_upper95  \
96196      20.9      13.09       0.0   India  ...                         NaN   
96197      1

### Forecast National Level

In [11]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]
daily_df = daily_df.drop(columns=['week_start'], errors='ignore')

# ✅ Aggregate to Country-level mean per day
country_df = (daily_df
              .groupby(['Country', 'time'])[climate_vars]
              .mean()
              .reset_index())

forecast_list = []

# ✅ Loop through each Country and forecast all variables
for country in country_df['Country'].unique():
    country_data = country_df[country_df['Country'] == country].copy()
    country_data.set_index('time', inplace=True)

    last_date = country_data.index.max()
    country_forecasts = {'Country': [], 'time': []}

    for var in climate_vars:
        series = country_data[var].dropna()

        if len(series) > 30:
            try:
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                if var == climate_vars[0]:
                    country_forecasts['Country'] = [country] * 7
                    country_forecasts['time'] = future_dates

                country_forecasts[var] = forecast.values
                country_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                country_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {country}: {e}")
                country_forecasts[var] = [None] * 7
                country_forecasts[f"{var}_lower95"] = [None] * 7
                country_forecasts[f"{var}_upper95"] = [None] * 7

    forecast_df = pd.DataFrame(country_forecasts)
    forecast_list.append(forecast_df)

# ✅ Combine all country forecasts
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename forecast columns
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical aggregated data
combined_df_with_forecast = pd.merge(
    country_df,
    forecast_df_all,
    on=['Country', 'time'],
    how='outer'
)

# ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Country', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("india_country_weather_with_prediction_2022_2025.csv", index=False)

# Preview
print(combined_df_with_forecast.tail(10))



     Country       time  temp_mean  rhum_mean  prcp_sum  wspd_mean  tsun_sum  \
3197   India 2025-10-03      25.24       81.6       4.5       8.98       0.0   
3198   India 2025-10-04      25.06      81.39      5.03       8.19       0.0   
3199   India 2025-10-05       24.7      80.32      7.23       8.18       0.0   
3200   India 2025-10-06       <NA>       <NA>      <NA>       <NA>      <NA>   
3201   India 2025-10-07       <NA>       <NA>      <NA>       <NA>      <NA>   
3202   India 2025-10-08       <NA>       <NA>      <NA>       <NA>      <NA>   
3203   India 2025-10-09       <NA>       <NA>      <NA>       <NA>      <NA>   
3204   India 2025-10-10       <NA>       <NA>      <NA>       <NA>      <NA>   
3205   India 2025-10-11       <NA>       <NA>      <NA>       <NA>      <NA>   
3206   India 2025-10-12       <NA>       <NA>      <NA>       <NA>      <NA>   

      temp_mean_forecast  temp_mean_forecast_lower95  \
3197                 NaN                         NaN   
3198   

## Thailand


### Daily Historical Data

In [12]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import numpy as np

# Define time period
start = datetime(2017, 1, 1)
end = datetime(2025, 12, 31, 23, 59)

# Define major administrative cities/divisions in Thailand with approximate coordinates
thailand_provinces = {
    'Bangkok': Point(13.7563, 100.5018, 1),
    'Amnat Charoen': Point(15.8655, 104.6258, 150),
    'Ang Thong': Point(14.5896, 100.4550, 10),
    'Bueng Kan': Point(18.3609, 103.6465, 160),
    'Buriram': Point(14.9930, 103.1029, 163),
    'Chachoengsao': Point(13.6904, 101.0779, 4),
    'Chai Nat': Point(15.1850, 100.1251, 20),
    'Chaiyaphum': Point(15.8042, 102.0376, 185),
    'Chanthaburi': Point(12.6112, 102.1030, 10),
    'Chiang Mai': Point(18.7883, 98.9853, 310),
    'Chiang Rai': Point(19.9105, 99.8406, 390),
    'Chonburi': Point(13.3611, 100.9847, 15),
    'Chumphon': Point(10.4931, 99.1800, 45),
    'Kalasin': Point(16.4322, 103.5064, 152),
    'Kamphaeng Phet': Point(16.4834, 99.5222, 111),
    'Kanchanaburi': Point(14.0176, 99.5228, 31),
    'Khon Kaen': Point(16.4419, 102.8350, 167),
    'Krabi': Point(8.0863, 98.9063, 12),
    'Lampang': Point(18.2888, 99.4909, 225),
    'Lamphun': Point(18.5804, 99.0079, 250),
    'Loei': Point(17.4905, 101.7275, 260),
    'Lopburi': Point(14.7995, 100.6534, 20),
    'Mae Hong Son': Point(19.3020, 97.9654, 250),
    'Maha Sarakham': Point(16.1849, 103.2991, 145),
    'Mukdahan': Point(16.5453, 104.7235, 160),
    'Nakhon Nayok': Point(14.2046, 101.2131, 25),
    'Nakhon Pathom': Point(13.8199, 100.0621, 10),
    'Nakhon Phanom': Point(17.4088, 104.7784, 160),
    'Nakhon Ratchasima': Point(14.9799, 102.0977, 210),
    'Nakhon Sawan': Point(15.7047, 100.1372, 36),
    'Nakhon Si Thammarat': Point(8.4304, 99.9631, 15),
    'Nan': Point(18.7833, 100.7833, 218),
    'Narathiwat': Point(6.4264, 101.8253, 7),
    'Nong Bua Lamphu': Point(17.2046, 102.4407, 220),
    'Nong Khai': Point(17.8783, 102.7413, 180),
    'Nonthaburi': Point(13.8591, 100.5217, 5),
    'Pathum Thani': Point(14.0208, 100.5250, 3),
    'Pattani': Point(6.8691, 101.2501, 3),
    'Phang Nga': Point(8.4500, 98.5330, 10),
    'Phatthalung': Point(7.6175, 100.0779, 5),
    'Phayao': Point(19.1632, 99.9010, 371),
    'Phetchabun': Point(16.4190, 101.1606, 114),
    'Phetchaburi': Point(13.1117, 99.9447, 12),
    'Phichit': Point(16.4419, 100.3488, 40),
    'Phitsanulok': Point(16.8281, 100.2729, 45),
    'Phrae': Point(18.1446, 100.1403, 165),
    'Phuket': Point(7.8804, 98.3923, 5),
    'Prachinburi': Point(14.0457, 101.3730, 30),
    'Prachuap Khiri Khan': Point(11.8130, 99.7967, 11),
    'Ranong': Point(9.9529, 98.6085, 10),
    'Ratchaburi': Point(13.5367, 99.8171, 10),
    'Rayong': Point(12.6814, 101.2789, 9),
    'Roi Et': Point(16.0567, 103.6537, 130),
    'Sa Kaeo': Point(13.8240, 102.0646, 50),
    'Sakon Nakhon': Point(17.1545, 104.1476, 175),
    'Samut Prakan': Point(13.5991, 100.5968, 2),
    'Samut Sakhon': Point(13.5470, 100.2744, 2),
    'Samut Songkhram': Point(13.4146, 100.0023, 2),
    'Saraburi': Point(14.5289, 100.9106, 20),
    'Satun': Point(6.6238, 100.0668, 5),
    'Sing Buri': Point(14.8879, 100.4040, 9),
    'Sisaket': Point(15.1186, 104.3220, 130),
    'Songkhla': Point(7.1897, 100.5950, 15),
    'Sukhothai': Point(17.0078, 99.8230, 60),
    'Suphan Buri': Point(14.4745, 100.1177, 9),
    'Surat Thani': Point(9.1400, 99.3332, 6),
    'Surin': Point(14.8818, 103.4936, 155),
    'Tak': Point(16.8690, 99.1263, 135),
    'Trang': Point(7.5590, 99.6113, 7),
    'Trat': Point(12.2428, 102.5180, 10),
    'Ubon Ratchathani': Point(15.2448, 104.8473, 120),
    'Udon Thani': Point(17.4138, 102.7872, 180),
    'Uthai Thani': Point(15.3794, 100.0245, 37),
    'Uttaradit': Point(17.6208, 100.0995, 65),
    'Yala': Point(6.5419, 101.2800, 9),
    'Yasothon': Point(15.7941, 104.1453, 127)
}

# Initialize dictionary to store DataFrames
all_data = {}

# ✅ Function to calculate Relative Humidity using Magnus formula
def calc_rh(temp, dewpt):
    temp = np.array(temp, dtype=float)
    dewpt = np.array(dewpt, dtype=float)
    mask = (~np.isnan(temp)) & (~np.isnan(dewpt))
    rh = np.full(temp.shape, np.nan)
    rh[mask] = 100 * np.exp((17.625 * dewpt[mask]) / (243.04 + dewpt[mask])) / \
                      np.exp((17.625 * temp[mask]) / (243.04 + temp[mask]))
    return rh

# ✅ Fetch hourly data for each division
for division, point in thailand_provinces.items():
    try:
        print(f"Fetching hourly data for {division}...")
        data = Hourly(point, start, end).fetch()

        # Skip if no data
        if data.empty:
            print(f"No data available for {division}")
            continue

        data['Division'] = division

        # Calculate RH if missing
        if 'rhum' not in data.columns and 'temp' in data.columns and 'dwpt' in data.columns:
            data['rhum'] = calc_rh(data['temp'].values, data['dwpt'].values)

        all_data[division] = data

    except Exception as e:
        print(f"Failed to fetch data for {division}: {e}")

# ✅ Combine all data into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data.values()).reset_index()
else:
    combined_df = pd.DataFrame()
    print("No data fetched for any division.")

# ✅ Ensure 'time' is datetime
if not combined_df.empty:
    combined_df['time'] = pd.to_datetime(combined_df['time'])

    # ✅ Group by Division and resample to daily
    daily_df = combined_df.groupby('Division').resample('D', on='time').agg({
        'temp': ['mean', 'max', 'min'],   # Avg, Max, Min temperature
        'rhum': 'mean',                   # Avg relative humidity
        'prcp': 'sum',                    # Total daily precipitation
        'wspd': 'mean',                   # Avg wind speed
        'tsun': 'sum'                     # Sum of sun exposure
    }).reset_index()

    # ✅ Flatten multi-level columns
    daily_df.columns = ['Division', 'time', 'temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    # ✅ Add 'Country' column
    daily_df['Country'] = 'Thailand'

     # ✅ Create provisional columns (copy today & future values)
    today = datetime.now().date()
    climate_vars = ['temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    for col in climate_vars:
        new_col = f"{col}_provisional"
        # Copy today's and future values into _provisional
        daily_df[new_col] = np.where(daily_df['time'].dt.date >= today, daily_df[col], np.nan)

        # Clear only future values from original columns, keep today intact
        daily_df.loc[daily_df['time'].dt.date > today, col] = np.nan

    # ✅ Round and save
    daily_df = daily_df.round(2)

    # ✅ Save to CSV
    daily_df.to_csv("thailand_states_weather_daily_2022_2025.csv", index=False)

    # ✅ Check last few rows
    print(daily_df.tail())
else:
    print("Daily DataFrame is empty.")

Fetching hourly data for Bangkok...
Fetching hourly data for Amnat Charoen...
Failed to fetch data for Amnat Charoen: 'Hourly' object has no attribute '_types'
Fetching hourly data for Ang Thong...
Fetching hourly data for Bueng Kan...
Fetching hourly data for Buriram...
Fetching hourly data for Chachoengsao...
Fetching hourly data for Chai Nat...
Fetching hourly data for Chaiyaphum...
Fetching hourly data for Chanthaburi...
Fetching hourly data for Chiang Mai...
Fetching hourly data for Chiang Rai...
Fetching hourly data for Chonburi...
Fetching hourly data for Chumphon...


Fetching hourly data for Kalasin...
Fetching hourly data for Kamphaeng Phet...
Fetching hourly data for Kanchanaburi...
Fetching hourly data for Khon Kaen...
Fetching hourly data for Krabi...
Fetching hourly data for Lampang...
Fetching hourly data for Lamphun...
Fetching hourly data for Loei...
Fetching hourly data for Lopburi...
Fetching hourly data for Mae Hong Son...
Fetching hourly data for Maha Sarakham...
Fetching hourly data for Mukdahan...
Fetching hourly data for Nakhon Nayok...
Fetching hourly data for Nakhon Pathom...
Fetching hourly data for Nakhon Phanom...
Fetching hourly data for Nakhon Ratchasima...
Fetching hourly data for Nakhon Sawan...
Fetching hourly data for Nakhon Si Thammarat...
Fetching hourly data for Nan...
Fetching hourly data for Narathiwat...
Fetching hourly data for Nong Bua Lamphu...
Fetching hourly data for Nong Khai...
Fetching hourly data for Nonthaburi...
Fetching hourly data for Pathum Thani...
Fetching hourly data for Pattani...
Fetching hourly da

Fetching hourly data for Ratchaburi...
Fetching hourly data for Rayong...


Fetching hourly data for Roi Et...
Fetching hourly data for Sa Kaeo...
Fetching hourly data for Sakon Nakhon...
Fetching hourly data for Samut Prakan...
Fetching hourly data for Samut Sakhon...
Failed to fetch data for Samut Sakhon: 'Hourly' object has no attribute '_types'
Fetching hourly data for Samut Songkhram...
Fetching hourly data for Saraburi...
Failed to fetch data for Saraburi: 'Hourly' object has no attribute '_types'
Fetching hourly data for Satun...
Fetching hourly data for Sing Buri...
Fetching hourly data for Sisaket...
Fetching hourly data for Songkhla...
Fetching hourly data for Sukhothai...
Fetching hourly data for Suphan Buri...
Fetching hourly data for Surat Thani...
Fetching hourly data for Surin...
Fetching hourly data for Tak...
Fetching hourly data for Trang...
Fetching hourly data for Trat...
Failed to fetch data for Trat: 'Hourly' object has no attribute '_types'
Fetching hourly data for Ubon Ratchathani...
Fetching hourly data for Udon Thani...
Fetching hourl

### Forecast

In [13]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data before forecasting
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]

forecast_list = []

# ✅ Loop through each Division and forecast all variables
for division in daily_df['Division'].unique():
    division_data = daily_df[daily_df['Division'] == division].copy()
    division_data.set_index('time', inplace=True)

    # Get the last confirmed date for this division
    last_date = division_data.index.max()

    # Temporary dict to store forecasts for this division
    division_forecasts = {'Division': [], 'time': []}

    for var in climate_vars:
        series = division_data[var].dropna()

        if len(series) > 30:
            try:
                # Fit SARIMAX model
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                # ✅ Forecast next 7 days
                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                # Initialize division & dates on first variable
                if var == climate_vars[0]:
                    division_forecasts['Division'] = [division] * 7
                    division_forecasts['time'] = future_dates

                # Add forecast and CI columns
                division_forecasts[var] = forecast.values
                division_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                division_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {division}: {e}")
                division_forecasts[var] = [None] * 7
                division_forecasts[f"{var}_lower95"] = [None] * 7
                division_forecasts[f"{var}_upper95"] = [None] * 7

    # Create DataFrame for this division forecast
    forecast_df = pd.DataFrame(division_forecasts)
    forecast_df['Country'] = 'Thailand'
    forecast_list.append(forecast_df)

# ✅ Combine forecasts for all divisions
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename columns to avoid overwriting historical values
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical daily_df
combined_df_with_forecast = pd.merge(
    daily_df,
    forecast_df_all,
    on=['Division', 'time', 'Country'],
    how='outer'
)

# ✅ Ensure datetime for merged DataFrame
combined_df_with_forecast['time'] = pd.to_datetime(combined_df_with_forecast['time'])

    # ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

 # ✅ Remove all *_provisional columns
combined_df_with_forecast = combined_df_with_forecast.drop(
        columns=[col for col in combined_df_with_forecast.columns if col.endswith('_provisional')],
        errors='ignore'
    )

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Division', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("thailand_states_weather_with_prediction_2022_2025.csv", index=False)

# Preview last rows
print(combined_df_with_forecast.tail(10))

       Division       time  temp_mean  temp_max  temp_min  rhum_mean  \
224480     Yala 2025-10-03      24.81      30.2      22.1      90.33   
224481     Yala 2025-10-04      26.14      31.4      22.3       85.5   
224482     Yala 2025-10-05      26.38      32.0      23.0       86.5   
224483     Yala 2025-10-06       <NA>      <NA>      <NA>       <NA>   
224484     Yala 2025-10-07       <NA>      <NA>      <NA>       <NA>   
224485     Yala 2025-10-08       <NA>      <NA>      <NA>       <NA>   
224486     Yala 2025-10-09       <NA>      <NA>      <NA>       <NA>   
224487     Yala 2025-10-10       <NA>      <NA>      <NA>       <NA>   
224488     Yala 2025-10-11       <NA>      <NA>      <NA>       <NA>   
224489     Yala 2025-10-12       <NA>      <NA>      <NA>       <NA>   

        prcp_sum  wspd_mean  tsun_sum   Country  ...  \
224480      19.8       6.08       0.0  Thailand  ...   
224481      16.2       7.48       0.0  Thailand  ...   
224482       5.5       6.23       0.0  

### Forecast National Level

In [14]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]
daily_df = daily_df.drop(columns=['week_start'], errors='ignore')

# ✅ Aggregate to Country-level mean per day
country_df = (daily_df
              .groupby(['Country', 'time'])[climate_vars]
              .mean()
              .reset_index())

forecast_list = []

# ✅ Loop through each Country and forecast all variables
for country in country_df['Country'].unique():
    country_data = country_df[country_df['Country'] == country].copy()
    country_data.set_index('time', inplace=True)

    last_date = country_data.index.max()
    country_forecasts = {'Country': [], 'time': []}

    for var in climate_vars:
        series = country_data[var].dropna()

        if len(series) > 30:
            try:
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                if var == climate_vars[0]:
                    country_forecasts['Country'] = [country] * 7
                    country_forecasts['time'] = future_dates

                country_forecasts[var] = forecast.values
                country_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                country_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {country}: {e}")
                country_forecasts[var] = [None] * 7
                country_forecasts[f"{var}_lower95"] = [None] * 7
                country_forecasts[f"{var}_upper95"] = [None] * 7

    forecast_df = pd.DataFrame(country_forecasts)
    forecast_list.append(forecast_df)

# ✅ Combine all country forecasts
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename forecast columns
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical aggregated data
combined_df_with_forecast = pd.merge(
    country_df,
    forecast_df_all,
    on=['Country', 'time'],
    how='outer'
)

# ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Country', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("thailand_country_weather_with_prediction_2022_2025.csv", index=False)

# Preview
print(combined_df_with_forecast.tail(10))

       Country       time  temp_mean  rhum_mean  prcp_sum  wspd_mean  \
3197  Thailand 2025-10-03      26.97      85.26      9.37       6.29   
3198  Thailand 2025-10-04      26.53      87.03      5.22       5.78   
3199  Thailand 2025-10-05      27.11      83.09      3.53        8.4   
3200  Thailand 2025-10-06       <NA>       <NA>      <NA>       <NA>   
3201  Thailand 2025-10-07       <NA>       <NA>      <NA>       <NA>   
3202  Thailand 2025-10-08       <NA>       <NA>      <NA>       <NA>   
3203  Thailand 2025-10-09       <NA>       <NA>      <NA>       <NA>   
3204  Thailand 2025-10-10       <NA>       <NA>      <NA>       <NA>   
3205  Thailand 2025-10-11       <NA>       <NA>      <NA>       <NA>   
3206  Thailand 2025-10-12       <NA>       <NA>      <NA>       <NA>   

      tsun_sum  temp_mean_forecast  temp_mean_forecast_lower95  \
3197       0.0                 NaN                         NaN   
3198       0.0                 NaN                         NaN   
3199     

## Sri Lanka

### Daily Historical Data

In [15]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import numpy as np

# Define time period
start = datetime(2017, 1, 1)
end = datetime(2025, 12, 31, 23, 59)

# Define Sri Lanka provinces with approximate coordinates (based on provincial capitals)
sri_lanka_provinces = {
    'Central': Point(7.2906, 80.6337, 488),        # Kandy
    'Eastern': Point(7.7338, 81.6986, 16),         # Trincomalee
    'North Central': Point(8.3114, 80.4037, 80),   # Anuradhapura
    'Northern': Point(9.6626, 80.0255, 5),         # Jaffna
    'North Western': Point(7.7662, 80.0048, 55),   # Kurunegala
    'Sabaragamuwa': Point(6.6828, 80.3990, 120),   # Ratnapura
    'Southern': Point(6.0535, 80.2210, 10),        # Galle
    'Uva': Point(6.8410, 81.0036, 970),            # Badulla
    'Western': Point(6.9271, 79.8612, 5),          # Colombo
}

# Initialize dictionary to store DataFrames
all_data = {}

# ✅ Function to calculate Relative Humidity using Magnus formula
def calc_rh(temp, dewpt):
    temp = np.array(temp, dtype=float)
    dewpt = np.array(dewpt, dtype=float)
    mask = (~np.isnan(temp)) & (~np.isnan(dewpt))
    rh = np.full(temp.shape, np.nan)
    rh[mask] = 100 * np.exp((17.625 * dewpt[mask]) / (243.04 + dewpt[mask])) / \
                      np.exp((17.625 * temp[mask]) / (243.04 + temp[mask]))
    return rh

# ✅ Fetch hourly data for each division
for division, point in sri_lanka_provinces.items():
    try:
        print(f"Fetching hourly data for {division}...")
        data = Hourly(point, start, end).fetch()

        # Skip if no data
        if data.empty:
            print(f"No data available for {division}")
            continue

        data['Division'] = division

        # Calculate RH if missing
        if 'rhum' not in data.columns and 'temp' in data.columns and 'dwpt' in data.columns:
            data['rhum'] = calc_rh(data['temp'].values, data['dwpt'].values)

        all_data[division] = data

    except Exception as e:
        print(f"Failed to fetch data for {division}: {e}")

# ✅ Combine all data into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data.values()).reset_index()
else:
    combined_df = pd.DataFrame()
    print("No data fetched for any division.")

# ✅ Ensure 'time' is datetime
if not combined_df.empty:
    combined_df['time'] = pd.to_datetime(combined_df['time'])

    # ✅ Group by Division and resample to daily
    daily_df = combined_df.groupby('Division').resample('D', on='time').agg({
        'temp': ['mean', 'max', 'min'],   # Avg, Max, Min temperature
        'rhum': 'mean',                   # Avg relative humidity
        'prcp': 'sum',                    # Total daily precipitation
        'wspd': 'mean',                   # Avg wind speed
        'tsun': 'sum'                     # Sum of sun exposure
    }).reset_index()

    # ✅ Flatten multi-level columns
    daily_df.columns = ['Division', 'time', 'temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    # ✅ Add 'Country' column
    daily_df['Country'] = 'Sri Lanka'

     # ✅ Create provisional columns (copy today & future values)
    today = datetime.now().date()
    climate_vars = ['temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    for col in climate_vars:
        new_col = f"{col}_provisional"
        # Copy today's and future values into _provisional
        daily_df[new_col] = np.where(daily_df['time'].dt.date >= today, daily_df[col], np.nan)

        # Clear only future values from original columns, keep today intact
        daily_df.loc[daily_df['time'].dt.date > today, col] = np.nan

    # ✅ Round and save
    daily_df = daily_df.round(2)

    # ✅ Save to CSV
    daily_df.to_csv("sri_lanka_states_weather_daily_2022_2025.csv", index=False)

    # ✅ Check last few rows
    print(daily_df.tail())
else:
    print("Daily DataFrame is empty.")

Fetching hourly data for Central...
Failed to fetch data for Central: 'Hourly' object has no attribute '_types'
Fetching hourly data for Eastern...
Fetching hourly data for North Central...
Fetching hourly data for Northern...


Fetching hourly data for North Western...
Failed to fetch data for North Western: 'Hourly' object has no attribute '_types'
Fetching hourly data for Sabaragamuwa...
Failed to fetch data for Sabaragamuwa: 'Hourly' object has no attribute '_types'
Fetching hourly data for Southern...
Failed to fetch data for Southern: 'Hourly' object has no attribute '_types'
Fetching hourly data for Uva...
Failed to fetch data for Uva: 'Hourly' object has no attribute '_types'
Fetching hourly data for Western...


      Division       time  temp_mean  temp_max  temp_min  rhum_mean  prcp_sum  \
10660  Western 2025-10-11       <NA>      <NA>      <NA>       <NA>      <NA>   
10661  Western 2025-10-12       <NA>      <NA>      <NA>       <NA>      <NA>   
10662  Western 2025-10-13       <NA>      <NA>      <NA>       <NA>      <NA>   
10663  Western 2025-10-14       <NA>      <NA>      <NA>       <NA>      <NA>   
10664  Western 2025-10-15       <NA>      <NA>      <NA>       <NA>      <NA>   

       wspd_mean  tsun_sum    Country  temp_mean_provisional  \
10660       <NA>      <NA>  Sri Lanka                  27.72   
10661       <NA>      <NA>  Sri Lanka                  27.42   
10662       <NA>      <NA>  Sri Lanka                  27.45   
10663       <NA>      <NA>  Sri Lanka                  27.38   
10664       <NA>      <NA>  Sri Lanka                  28.16   

       temp_max_provisional  temp_min_provisional  rhum_mean_provisional  \
10660                  30.2                  25.6   

### Forecast

In [16]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data before forecasting
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]

forecast_list = []

# ✅ Loop through each Division and forecast all variables
for division in daily_df['Division'].unique():
    division_data = daily_df[daily_df['Division'] == division].copy()
    division_data.set_index('time', inplace=True)

    # Get the last confirmed date for this division
    last_date = division_data.index.max()

    # Temporary dict to store forecasts for this division
    division_forecasts = {'Division': [], 'time': []}

    for var in climate_vars:
        series = division_data[var].dropna()

        if len(series) > 30:
            try:
                # Fit SARIMAX model
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                # ✅ Forecast next 7 days
                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                # Initialize division & dates on first variable
                if var == climate_vars[0]:
                    division_forecasts['Division'] = [division] * 7
                    division_forecasts['time'] = future_dates

                # Add forecast and CI columns
                division_forecasts[var] = forecast.values
                division_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                division_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {division}: {e}")
                division_forecasts[var] = [None] * 7
                division_forecasts[f"{var}_lower95"] = [None] * 7
                division_forecasts[f"{var}_upper95"] = [None] * 7

    # Create DataFrame for this division forecast
    forecast_df = pd.DataFrame(division_forecasts)
    forecast_df['Country'] = 'Sri Lanka'
    forecast_list.append(forecast_df)

# ✅ Combine forecasts for all divisions
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename columns to avoid overwriting historical values
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical daily_df
combined_df_with_forecast = pd.merge(
    daily_df,
    forecast_df_all,
    on=['Division', 'time', 'Country'],
    how='outer'
)

# ✅ Ensure datetime for merged DataFrame
combined_df_with_forecast['time'] = pd.to_datetime(combined_df_with_forecast['time'])

    # ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

 # ✅ Remove all *_provisional columns
combined_df_with_forecast = combined_df_with_forecast.drop(
        columns=[col for col in combined_df_with_forecast.columns if col.endswith('_provisional')],
        errors='ignore'
    )

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Division', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("sri_lanka_states_weather_with_prediction_2022_2025.csv", index=False)

# Preview last rows
print(combined_df_with_forecast.tail(10))

      Division       time  temp_mean  temp_max  temp_min  rhum_mean  prcp_sum  \
10643  Western 2025-10-03       28.7      30.8      26.6      79.38       0.0   
10644  Western 2025-10-04      28.25      30.2      26.4      81.83       8.5   
10645  Western 2025-10-05      27.55      29.3      25.7      80.42       5.4   
10646  Western 2025-10-06       <NA>      <NA>      <NA>       <NA>      <NA>   
10647  Western 2025-10-07       <NA>      <NA>      <NA>       <NA>      <NA>   
10648  Western 2025-10-08       <NA>      <NA>      <NA>       <NA>      <NA>   
10649  Western 2025-10-09       <NA>      <NA>      <NA>       <NA>      <NA>   
10650  Western 2025-10-10       <NA>      <NA>      <NA>       <NA>      <NA>   
10651  Western 2025-10-11       <NA>      <NA>      <NA>       <NA>      <NA>   
10652  Western 2025-10-12       <NA>      <NA>      <NA>       <NA>      <NA>   

       wspd_mean  tsun_sum    Country  ...  rhum_mean_forecast_upper95  \
10643       3.35       0.0  Sri La

### Forecast National Level

In [17]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]
daily_df = daily_df.drop(columns=['week_start'], errors='ignore')

# ✅ Aggregate to Country-level mean per day
country_df = (daily_df
              .groupby(['Country', 'time'])[climate_vars]
              .mean()
              .reset_index())

forecast_list = []

# ✅ Loop through each Country and forecast all variables
for country in country_df['Country'].unique():
    country_data = country_df[country_df['Country'] == country].copy()
    country_data.set_index('time', inplace=True)

    last_date = country_data.index.max()
    country_forecasts = {'Country': [], 'time': []}

    for var in climate_vars:
        series = country_data[var].dropna()

        if len(series) > 30:
            try:
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                if var == climate_vars[0]:
                    country_forecasts['Country'] = [country] * 7
                    country_forecasts['time'] = future_dates

                country_forecasts[var] = forecast.values
                country_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                country_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {country}: {e}")
                country_forecasts[var] = [None] * 7
                country_forecasts[f"{var}_lower95"] = [None] * 7
                country_forecasts[f"{var}_upper95"] = [None] * 7

    forecast_df = pd.DataFrame(country_forecasts)
    forecast_list.append(forecast_df)

# ✅ Combine all country forecasts
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename forecast columns
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical aggregated data
combined_df_with_forecast = pd.merge(
    country_df,
    forecast_df_all,
    on=['Country', 'time'],
    how='outer'
)

# ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Country', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("sri_lanka_country_weather_with_prediction_2022_2025.csv", index=False)

# Preview
print(combined_df_with_forecast.tail(10))

        Country       time  temp_mean  rhum_mean  prcp_sum  wspd_mean  \
3197  Sri Lanka 2025-10-03      28.62      77.08      2.52       11.0   
3198  Sri Lanka 2025-10-04      27.68      81.73      9.12        9.3   
3199  Sri Lanka 2025-10-05      27.62      80.31      4.05      10.87   
3200  Sri Lanka 2025-10-06       <NA>       <NA>      <NA>       <NA>   
3201  Sri Lanka 2025-10-07       <NA>       <NA>      <NA>       <NA>   
3202  Sri Lanka 2025-10-08       <NA>       <NA>      <NA>       <NA>   
3203  Sri Lanka 2025-10-09       <NA>       <NA>      <NA>       <NA>   
3204  Sri Lanka 2025-10-10       <NA>       <NA>      <NA>       <NA>   
3205  Sri Lanka 2025-10-11       <NA>       <NA>      <NA>       <NA>   
3206  Sri Lanka 2025-10-12       <NA>       <NA>      <NA>       <NA>   

      tsun_sum  temp_mean_forecast  temp_mean_forecast_lower95  \
3197       0.0                 NaN                         NaN   
3198       0.0                 NaN                         NaN  

## Myanmar

### Daily Historical Data

In [18]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import numpy as np

# Define time period
start = datetime(2017, 1, 1)
end = datetime(2025, 12, 31, 23, 59)

# Define Myanmar's 14 states and regions using their administrative capitals
myanmar_regions = {
    'Ayeyarwady': Point(16.7792, 94.7321, 5),       # Pathein
    'Bago': Point(17.3365, 96.5011, 20),            # Bago
    'Chin': Point(22.1964, 93.5813, 860),           # Hakha
    'Kachin': Point(25.3914, 97.3936, 144),         # Myitkyina
    'Kayah': Point(19.3022, 97.9654, 880),          # Loikaw
    'Kayin': Point(16.8661, 97.6319, 18),           # Hpa-An
    'Magway': Point(20.1500, 94.9167, 76),          # Magway
    'Mandalay': Point(21.9588, 96.0891, 77),        # Mandalay
    'Mon': Point(16.4540, 97.6430, 21),             # Mawlamyine
    'Naypyidaw': Point(19.7633, 96.0785, 115),      # Union Territory
    'Rakhine': Point(20.1521, 92.8960, 20),         # Sittwe
    'Sagaing': Point(22.1700, 95.1400, 70),         # Sagaing
    'Shan': Point(20.9017, 97.0380, 1070),          # Taunggyi
    'Tanintharyi': Point(12.4563, 99.1830, 10),     # Dawei
        # Optional: Yangon Region can be included separately if needed
    'Yangon': Point(16.8409, 96.1735, 15)           # Yangon
}

# Initialize dictionary to store DataFrames
all_data = {}

# ✅ Function to calculate Relative Humidity using Magnus formula
def calc_rh(temp, dewpt):
    temp = np.array(temp, dtype=float)
    dewpt = np.array(dewpt, dtype=float)
    mask = (~np.isnan(temp)) & (~np.isnan(dewpt))
    rh = np.full(temp.shape, np.nan)
    rh[mask] = 100 * np.exp((17.625 * dewpt[mask]) / (243.04 + dewpt[mask])) / \
                      np.exp((17.625 * temp[mask]) / (243.04 + temp[mask]))
    return rh

# ✅ Fetch hourly data for each division
for division, point in myanmar_regions.items():
    try:
        print(f"Fetching hourly data for {division}...")
        data = Hourly(point, start, end).fetch()

        # Skip if no data
        if data.empty:
            print(f"No data available for {division}")
            continue

        data['Division'] = division

        # Calculate RH if missing
        if 'rhum' not in data.columns and 'temp' in data.columns and 'dwpt' in data.columns:
            data['rhum'] = calc_rh(data['temp'].values, data['dwpt'].values)

        all_data[division] = data

    except Exception as e:
        print(f"Failed to fetch data for {division}: {e}")

# ✅ Combine all data into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data.values()).reset_index()
else:
    combined_df = pd.DataFrame()
    print("No data fetched for any division.")

# ✅ Ensure 'time' is datetime
if not combined_df.empty:
    combined_df['time'] = pd.to_datetime(combined_df['time'])

    # ✅ Group by Division and resample to daily
    daily_df = combined_df.groupby('Division').resample('D', on='time').agg({
        'temp': ['mean', 'max', 'min'],   # Avg, Max, Min temperature
        'rhum': 'mean',                   # Avg relative humidity
        'prcp': 'sum',                    # Total daily precipitation
        'wspd': 'mean',                   # Avg wind speed
        'tsun': 'sum'                     # Sum of sun exposure
    }).reset_index()

    # ✅ Flatten multi-level columns
    daily_df.columns = ['Division', 'time', 'temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    # ✅ Add 'Country' column
    daily_df['Country'] = 'Myanmar'

     # ✅ Create provisional columns (copy today & future values)
    today = datetime.now().date()
    climate_vars = ['temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    for col in climate_vars:
        new_col = f"{col}_provisional"
        # Copy today's and future values into _provisional
        daily_df[new_col] = np.where(daily_df['time'].dt.date >= today, daily_df[col], np.nan)

        # Clear only future values from original columns, keep today intact
        daily_df.loc[daily_df['time'].dt.date > today, col] = np.nan

    # ✅ Round and save
    daily_df = daily_df.round(2)

    # ✅ Save to CSV
    daily_df.to_csv("myanmar_states_weather_daily_2022_2025.csv", index=False)

    # ✅ Check last few rows
    print(daily_df.tail())
else:
    print("Daily DataFrame is empty.")

Fetching hourly data for Ayeyarwady...
Fetching hourly data for Bago...
Failed to fetch data for Bago: 'Hourly' object has no attribute '_types'
Fetching hourly data for Chin...
Failed to fetch data for Chin: 'Hourly' object has no attribute '_types'
Fetching hourly data for Kachin...
Fetching hourly data for Kayah...
Failed to fetch data for Kayah: 'Hourly' object has no attribute '_types'
Fetching hourly data for Kayin...
Fetching hourly data for Magway...
Failed to fetch data for Magway: 'Hourly' object has no attribute '_types'
Fetching hourly data for Mandalay...
Fetching hourly data for Mon...
Fetching hourly data for Naypyidaw...


Fetching hourly data for Rakhine...
Fetching hourly data for Sagaing...
Failed to fetch data for Sagaing: 'Hourly' object has no attribute '_types'
Fetching hourly data for Shan...
Failed to fetch data for Shan: 'Hourly' object has no attribute '_types'
Fetching hourly data for Tanintharyi...
Failed to fetch data for Tanintharyi: 'Hourly' object has no attribute '_types'
Fetching hourly data for Yangon...
      Division       time  temp_mean  temp_max  temp_min  rhum_mean  prcp_sum  \
24563   Yangon 2025-10-10       <NA>      <NA>      <NA>       <NA>      <NA>   
24564   Yangon 2025-10-11       <NA>      <NA>      <NA>       <NA>      <NA>   
24565   Yangon 2025-10-12       <NA>      <NA>      <NA>       <NA>      <NA>   
24566   Yangon 2025-10-13       <NA>      <NA>      <NA>       <NA>      <NA>   
24567   Yangon 2025-10-14       <NA>      <NA>      <NA>       <NA>      <NA>   

       wspd_mean  tsun_sum  Country  temp_mean_provisional  \
24563       <NA>      <NA>  Myanmar       

### Forecast

In [19]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data before forecasting
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]

forecast_list = []

# ✅ Loop through each Division and forecast all variables
for division in daily_df['Division'].unique():
    division_data = daily_df[daily_df['Division'] == division].copy()
    division_data.set_index('time', inplace=True)

    # Get the last confirmed date for this division
    last_date = division_data.index.max()

    # Temporary dict to store forecasts for this division
    division_forecasts = {'Division': [], 'time': []}

    for var in climate_vars:
        series = division_data[var].dropna()

        if len(series) > 30:
            try:
                # Fit SARIMAX model
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                # ✅ Forecast next 7 days
                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                # Initialize division & dates on first variable
                if var == climate_vars[0]:
                    division_forecasts['Division'] = [division] * 7
                    division_forecasts['time'] = future_dates

                # Add forecast and CI columns
                division_forecasts[var] = forecast.values
                division_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                division_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {division}: {e}")
                division_forecasts[var] = [None] * 7
                division_forecasts[f"{var}_lower95"] = [None] * 7
                division_forecasts[f"{var}_upper95"] = [None] * 7

    # Create DataFrame for this division forecast
    forecast_df = pd.DataFrame(division_forecasts)
    forecast_df['Country'] = 'Myanmar'
    forecast_list.append(forecast_df)

# ✅ Combine forecasts for all divisions
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename columns to avoid overwriting historical values
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical daily_df
combined_df_with_forecast = pd.merge(
    daily_df,
    forecast_df_all,
    on=['Division', 'time', 'Country'],
    how='outer'
)

# ✅ Ensure datetime for merged DataFrame
combined_df_with_forecast['time'] = pd.to_datetime(combined_df_with_forecast['time'])

    # ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

 # ✅ Remove all *_provisional columns
combined_df_with_forecast = combined_df_with_forecast.drop(
        columns=[col for col in combined_df_with_forecast.columns if col.endswith('_provisional')],
        errors='ignore'
    )

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Division', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("myanmar_states_weather_with_prediction_2022_2025.csv", index=False)

# Preview last rows
print(combined_df_with_forecast.tail(10))

      Division       time  temp_mean  temp_max  temp_min  rhum_mean  prcp_sum  \
24538   Yangon 2025-10-03      28.18      32.0      25.0       86.5       0.0   
24539   Yangon 2025-10-04      27.99      32.0      25.4      86.25       0.0   
24540   Yangon 2025-10-05      27.92      31.0      25.4      84.38       0.0   
24541   Yangon 2025-10-06       <NA>      <NA>      <NA>       <NA>      <NA>   
24542   Yangon 2025-10-07       <NA>      <NA>      <NA>       <NA>      <NA>   
24543   Yangon 2025-10-08       <NA>      <NA>      <NA>       <NA>      <NA>   
24544   Yangon 2025-10-09       <NA>      <NA>      <NA>       <NA>      <NA>   
24545   Yangon 2025-10-10       <NA>      <NA>      <NA>       <NA>      <NA>   
24546   Yangon 2025-10-11       <NA>      <NA>      <NA>       <NA>      <NA>   
24547   Yangon 2025-10-12       <NA>      <NA>      <NA>       <NA>      <NA>   

       wspd_mean  tsun_sum  Country  ...  rhum_mean_forecast_upper95  \
24538       2.78       0.0  Myanmar 

### Forecast National Level

In [20]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]
daily_df = daily_df.drop(columns=['week_start'], errors='ignore')

# ✅ Aggregate to Country-level mean per day
country_df = (daily_df
              .groupby(['Country', 'time'])[climate_vars]
              .mean()
              .reset_index())

forecast_list = []

# ✅ Loop through each Country and forecast all variables
for country in country_df['Country'].unique():
    country_data = country_df[country_df['Country'] == country].copy()
    country_data.set_index('time', inplace=True)

    last_date = country_data.index.max()
    country_forecasts = {'Country': [], 'time': []}

    for var in climate_vars:
        series = country_data[var].dropna()

        if len(series) > 30:
            try:
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                if var == climate_vars[0]:
                    country_forecasts['Country'] = [country] * 7
                    country_forecasts['time'] = future_dates

                country_forecasts[var] = forecast.values
                country_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                country_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {country}: {e}")
                country_forecasts[var] = [None] * 7
                country_forecasts[f"{var}_lower95"] = [None] * 7
                country_forecasts[f"{var}_upper95"] = [None] * 7

    forecast_df = pd.DataFrame(country_forecasts)
    forecast_list.append(forecast_df)

# ✅ Combine all country forecasts
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename forecast columns
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical aggregated data
combined_df_with_forecast = pd.merge(
    country_df,
    forecast_df_all,
    on=['Country', 'time'],
    how='outer'
)

# ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Country', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("myanmar_country_weather_with_prediction_2022_2025.csv", index=False)

# Preview
print(combined_df_with_forecast.tail(10))

      Country       time  temp_mean  rhum_mean  prcp_sum  wspd_mean  tsun_sum  \
3197  Myanmar 2025-10-03       27.2      86.95      5.03       4.61       0.0   
3198  Myanmar 2025-10-04      27.05      87.58      9.52       5.11       0.0   
3199  Myanmar 2025-10-05      26.68      87.91      7.94       6.15       0.0   
3200  Myanmar 2025-10-06       <NA>       <NA>      <NA>       <NA>      <NA>   
3201  Myanmar 2025-10-07       <NA>       <NA>      <NA>       <NA>      <NA>   
3202  Myanmar 2025-10-08       <NA>       <NA>      <NA>       <NA>      <NA>   
3203  Myanmar 2025-10-09       <NA>       <NA>      <NA>       <NA>      <NA>   
3204  Myanmar 2025-10-10       <NA>       <NA>      <NA>       <NA>      <NA>   
3205  Myanmar 2025-10-11       <NA>       <NA>      <NA>       <NA>      <NA>   
3206  Myanmar 2025-10-12       <NA>       <NA>      <NA>       <NA>      <NA>   

      temp_mean_forecast  temp_mean_forecast_lower95  \
3197                 NaN                         NaN

## Bhutan

### Daily Historical Data

In [21]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import numpy as np

# Define time period
start = datetime(2017, 1, 1)
end = datetime(2025, 12, 31, 23, 59)

# Define approximate coordinates (district centers or major cities)
bhutan_districts = {
    'Thimphu': Point(27.4728, 89.6390, 2334),
    'Paro': Point(27.4305, 89.4167, 2195),
    'Punakha': Point(27.5916, 89.8771, 1250),
    'Wangdue Phodrang': Point(27.4333, 89.8833, 1300),
    'Haa': Point(27.3850, 89.2807, 2670),
    'Chukha': Point(26.8688, 89.5086, 1700),
    'Samdrup Jongkhar': Point(26.8006, 91.5036, 200),
    'Trashigang': Point(27.3333, 91.5500, 1070),
    'Mongar': Point(27.2667, 91.2500, 1600),
    'Lhuentse': Point(27.6667, 91.1833, 1700),
    'Bumthang': Point(27.5495, 90.7485, 2600),
    'Trongsa': Point(27.5026, 90.5083, 2200),
    'Zhemgang': Point(27.2167, 90.6667, 1900),
    'Sarpang': Point(26.8604, 90.2710, 400),
    'Tsirang': Point(27.0137, 90.1228, 1300),
    'Dagana': Point(27.0333, 89.8667, 1520),
    'Pemagatshel': Point(27.0333, 91.3000, 1300),
    'Samtse': Point(26.9000, 89.1000, 417),
    'Gasa': Point(27.9028, 89.7269, 2850),
    'Trashiyangtse': Point(27.6167, 91.5500, 1800)
}

# Initialize dictionary to store DataFrames
all_data = {}

# ✅ Function to calculate Relative Humidity using Magnus formula
def calc_rh(temp, dewpt):
    temp = np.array(temp, dtype=float)
    dewpt = np.array(dewpt, dtype=float)
    mask = (~np.isnan(temp)) & (~np.isnan(dewpt))
    rh = np.full(temp.shape, np.nan)
    rh[mask] = 100 * np.exp((17.625 * dewpt[mask]) / (243.04 + dewpt[mask])) / \
                      np.exp((17.625 * temp[mask]) / (243.04 + temp[mask]))
    return rh

# ✅ Fetch hourly data for each division
for division, point in bhutan_districts.items():
    try:
        print(f"Fetching hourly data for {division}...")
        data = Hourly(point, start, end).fetch()

        # Skip if no data
        if data.empty:
            print(f"No data available for {division}")
            continue

        data['Division'] = division

        # Calculate RH if missing
        if 'rhum' not in data.columns and 'temp' in data.columns and 'dwpt' in data.columns:
            data['rhum'] = calc_rh(data['temp'].values, data['dwpt'].values)

        all_data[division] = data

    except Exception as e:
        print(f"Failed to fetch data for {division}: {e}")

# ✅ Combine all data into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data.values()).reset_index()
else:
    combined_df = pd.DataFrame()
    print("No data fetched for any division.")

# ✅ Ensure 'time' is datetime
if not combined_df.empty:
    combined_df['time'] = pd.to_datetime(combined_df['time'])

    # ✅ Group by Division and resample to daily
    daily_df = combined_df.groupby('Division').resample('D', on='time').agg({
        'temp': ['mean', 'max', 'min'],   # Avg, Max, Min temperature
        'rhum': 'mean',                   # Avg relative humidity
        'prcp': 'sum',                    # Total daily precipitation
        'wspd': 'mean',                   # Avg wind speed
        'tsun': 'sum'                     # Sum of sun exposure
    }).reset_index()

    # ✅ Flatten multi-level columns
    daily_df.columns = ['Division', 'time', 'temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    # ✅ Add 'Country' column
    daily_df['Country'] = 'Bhutan'

     # ✅ Create provisional columns (copy today & future values)
    today = datetime.now().date()
    climate_vars = ['temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    for col in climate_vars:
        new_col = f"{col}_provisional"
        # Copy today's and future values into _provisional
        daily_df[new_col] = np.where(daily_df['time'].dt.date >= today, daily_df[col], np.nan)

        # Clear only future values from original columns, keep today intact
        daily_df.loc[daily_df['time'].dt.date > today, col] = np.nan

    # ✅ Round and save
    daily_df = daily_df.round(2)

    # ✅ Save to CSV
    daily_df.to_csv("bhutan_states_weather_daily_2022_2025.csv", index=False)

    # ✅ Check last few rows
    print(daily_df.tail())
else:
    print("Daily DataFrame is empty.")

Fetching hourly data for Thimphu...


Fetching hourly data for Paro...
Fetching hourly data for Punakha...
Failed to fetch data for Punakha: 'Hourly' object has no attribute '_types'
Fetching hourly data for Wangdue Phodrang...
Failed to fetch data for Wangdue Phodrang: 'Hourly' object has no attribute '_types'
Fetching hourly data for Haa...
Failed to fetch data for Haa: 'Hourly' object has no attribute '_types'
Fetching hourly data for Chukha...
Failed to fetch data for Chukha: 'Hourly' object has no attribute '_types'
Fetching hourly data for Samdrup Jongkhar...
Failed to fetch data for Samdrup Jongkhar: 'Hourly' object has no attribute '_types'
Fetching hourly data for Trashigang...
Failed to fetch data for Trashigang: 'Hourly' object has no attribute '_types'
Fetching hourly data for Mongar...
Failed to fetch data for Mongar: 'Hourly' object has no attribute '_types'
Fetching hourly data for Lhuentse...
Failed to fetch data for Lhuentse: 'Hourly' object has no attribute '_types'
Fetching hourly data for Bumthang...
Fa

### Forecast

In [22]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data before forecasting
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]

forecast_list = []

# ✅ Loop through each Division and forecast all variables
for division in daily_df['Division'].unique():
    division_data = daily_df[daily_df['Division'] == division].copy()
    division_data.set_index('time', inplace=True)

    # Get the last confirmed date for this division
    last_date = division_data.index.max()

    # Temporary dict to store forecasts for this division
    division_forecasts = {'Division': [], 'time': []}

    for var in climate_vars:
        series = division_data[var].dropna()

        if len(series) > 30:
            try:
                # Fit SARIMAX model
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                # ✅ Forecast next 7 days
                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                # Initialize division & dates on first variable
                if var == climate_vars[0]:
                    division_forecasts['Division'] = [division] * 7
                    division_forecasts['time'] = future_dates

                # Add forecast and CI columns
                division_forecasts[var] = forecast.values
                division_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                division_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {division}: {e}")
                division_forecasts[var] = [None] * 7
                division_forecasts[f"{var}_lower95"] = [None] * 7
                division_forecasts[f"{var}_upper95"] = [None] * 7

    # Create DataFrame for this division forecast
    forecast_df = pd.DataFrame(division_forecasts)
    forecast_df['Country'] = 'Bhutan'
    forecast_list.append(forecast_df)

# ✅ Combine forecasts for all divisions
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename columns to avoid overwriting historical values
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical daily_df
combined_df_with_forecast = pd.merge(
    daily_df,
    forecast_df_all,
    on=['Division', 'time', 'Country'],
    how='outer'
)

# ✅ Ensure datetime for merged DataFrame
combined_df_with_forecast['time'] = pd.to_datetime(combined_df_with_forecast['time'])

    # ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

 # ✅ Remove all *_provisional columns
combined_df_with_forecast = combined_df_with_forecast.drop(
        columns=[col for col in combined_df_with_forecast.columns if col.endswith('_provisional')],
        errors='ignore'
    )

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Division', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("bhutan_states_weather_with_prediction_2022_2025.csv", index=False)

# Preview last rows
print(combined_df_with_forecast.tail(10))

     Division       time  temp_mean  temp_max  temp_min  rhum_mean  prcp_sum  \
6402  Thimphu 2025-10-03      15.89      20.4      13.5      86.46       8.0   
6403  Thimphu 2025-10-04      16.02      21.4      13.4      85.04      21.5   
6404  Thimphu 2025-10-05      11.93      13.8      10.4      87.08      55.8   
6405  Thimphu 2025-10-06       <NA>      <NA>      <NA>       <NA>      <NA>   
6406  Thimphu 2025-10-07       <NA>      <NA>      <NA>       <NA>      <NA>   
6407  Thimphu 2025-10-08       <NA>      <NA>      <NA>       <NA>      <NA>   
6408  Thimphu 2025-10-09       <NA>      <NA>      <NA>       <NA>      <NA>   
6409  Thimphu 2025-10-10       <NA>      <NA>      <NA>       <NA>      <NA>   
6410  Thimphu 2025-10-11       <NA>      <NA>      <NA>       <NA>      <NA>   
6411  Thimphu 2025-10-12       <NA>      <NA>      <NA>       <NA>      <NA>   

      wspd_mean  tsun_sum Country  ...  rhum_mean_forecast_upper95  \
6402       6.13       0.0  Bhutan  ...           

### Forecast National Level

In [23]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]
daily_df = daily_df.drop(columns=['week_start'], errors='ignore')

# ✅ Aggregate to Country-level mean per day
country_df = (daily_df
              .groupby(['Country', 'time'])[climate_vars]
              .mean()
              .reset_index())

forecast_list = []

# ✅ Loop through each Country and forecast all variables
for country in country_df['Country'].unique():
    country_data = country_df[country_df['Country'] == country].copy()
    country_data.set_index('time', inplace=True)

    last_date = country_data.index.max()
    country_forecasts = {'Country': [], 'time': []}

    for var in climate_vars:
        series = country_data[var].dropna()

        if len(series) > 30:
            try:
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                if var == climate_vars[0]:
                    country_forecasts['Country'] = [country] * 7
                    country_forecasts['time'] = future_dates

                country_forecasts[var] = forecast.values
                country_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                country_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {country}: {e}")
                country_forecasts[var] = [None] * 7
                country_forecasts[f"{var}_lower95"] = [None] * 7
                country_forecasts[f"{var}_upper95"] = [None] * 7

    forecast_df = pd.DataFrame(country_forecasts)
    forecast_list.append(forecast_df)

# ✅ Combine all country forecasts
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename forecast columns
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical aggregated data
combined_df_with_forecast = pd.merge(
    country_df,
    forecast_df_all,
    on=['Country', 'time'],
    how='outer'
)

# ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Country', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("bhutan_country_weather_with_prediction_2022_2025.csv", index=False)

# Preview
print(combined_df_with_forecast.tail(10))

     Country       time  temp_mean  rhum_mean  prcp_sum  wspd_mean  tsun_sum  \
3196  Bhutan 2025-10-03      16.29      86.46       8.0       6.13       0.0   
3197  Bhutan 2025-10-04      16.42      85.04      21.5       3.95       0.0   
3198  Bhutan 2025-10-05      12.33      87.08      55.8        4.8       0.0   
3199  Bhutan 2025-10-06       <NA>       <NA>      <NA>       <NA>      <NA>   
3200  Bhutan 2025-10-07       <NA>       <NA>      <NA>       <NA>      <NA>   
3201  Bhutan 2025-10-08       <NA>       <NA>      <NA>       <NA>      <NA>   
3202  Bhutan 2025-10-09       <NA>       <NA>      <NA>       <NA>      <NA>   
3203  Bhutan 2025-10-10       <NA>       <NA>      <NA>       <NA>      <NA>   
3204  Bhutan 2025-10-11       <NA>       <NA>      <NA>       <NA>      <NA>   
3205  Bhutan 2025-10-12       <NA>       <NA>      <NA>       <NA>      <NA>   

      temp_mean_forecast  temp_mean_forecast_lower95  \
3196                 NaN                         NaN   
3197   

## Nepal

### Daily Historical Data

In [24]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import numpy as np

# Define time period
start = datetime(2017, 1, 1)
end = datetime(2025, 12, 31, 23, 59)

# Define Nepal's 7 provinces with representative coordinates (provincial capitals or central cities)
nepal_provinces = {
    'Province 1': Point(26.8120, 87.2846, 345),         # Biratnagar
    'Madhesh Province': Point(27.0000, 84.8667, 79),    # Birgunj
    'Bagmati Province': Point(27.7172, 85.3240, 1350),  # Kathmandu
    'Gandaki Province': Point(28.2096, 83.9856, 827),   # Pokhara
    'Lumbini Province': Point(27.6849, 83.4602, 150),   # Butwal
    'Karnali Province': Point(28.6000, 81.6333, 2130),  # Birendranagar (Surkhet)
    'Sudurpashchim Province': Point(28.7050, 80.5930, 200),  # Dhangadhi
}

# Initialize dictionary to store DataFrames
all_data = {}

# ✅ Function to calculate Relative Humidity using Magnus formula
def calc_rh(temp, dewpt):
    temp = np.array(temp, dtype=float)
    dewpt = np.array(dewpt, dtype=float)
    mask = (~np.isnan(temp)) & (~np.isnan(dewpt))
    rh = np.full(temp.shape, np.nan)
    rh[mask] = 100 * np.exp((17.625 * dewpt[mask]) / (243.04 + dewpt[mask])) / \
                      np.exp((17.625 * temp[mask]) / (243.04 + temp[mask]))
    return rh

# ✅ Fetch hourly data for each division
for division, point in nepal_provinces.items():
    try:
        print(f"Fetching hourly data for {division}...")
        data = Hourly(point, start, end).fetch()

        # Skip if no data
        if data.empty:
            print(f"No data available for {division}")
            continue

        data['Division'] = division

        # Calculate RH if missing
        if 'rhum' not in data.columns and 'temp' in data.columns and 'dwpt' in data.columns:
            data['rhum'] = calc_rh(data['temp'].values, data['dwpt'].values)

        all_data[division] = data

    except Exception as e:
        print(f"Failed to fetch data for {division}: {e}")

# ✅ Combine all data into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data.values()).reset_index()
else:
    combined_df = pd.DataFrame()
    print("No data fetched for any division.")

# ✅ Ensure 'time' is datetime
if not combined_df.empty:
    combined_df['time'] = pd.to_datetime(combined_df['time'])

    # ✅ Group by Division and resample to daily
    daily_df = combined_df.groupby('Division').resample('D', on='time').agg({
        'temp': ['mean', 'max', 'min'],   # Avg, Max, Min temperature
        'rhum': 'mean',                   # Avg relative humidity
        'prcp': 'sum',                    # Total daily precipitation
        'wspd': 'mean',                   # Avg wind speed
        'tsun': 'sum'                     # Sum of sun exposure
    }).reset_index()

    # ✅ Flatten multi-level columns
    daily_df.columns = ['Division', 'time', 'temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    # ✅ Add 'Country' column
    daily_df['Country'] = 'Nepal'

     # ✅ Create provisional columns (copy today & future values)
    today = datetime.now().date()
    climate_vars = ['temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    for col in climate_vars:
        new_col = f"{col}_provisional"
        # Copy today's and future values into _provisional
        daily_df[new_col] = np.where(daily_df['time'].dt.date >= today, daily_df[col], np.nan)

        # Clear only future values from original columns, keep today intact
        daily_df.loc[daily_df['time'].dt.date > today, col] = np.nan

    # ✅ Round and save
    daily_df = daily_df.round(2)

    # ✅ Save to CSV
    daily_df.to_csv("nepal_states_weather_daily_2022_2025.csv", index=False)

    # ✅ Check last few rows
    print(daily_df.tail())
else:
    print("Daily DataFrame is empty.")

Fetching hourly data for Province 1...
Failed to fetch data for Province 1: 'Hourly' object has no attribute '_types'
Fetching hourly data for Madhesh Province...


Fetching hourly data for Bagmati Province...
Fetching hourly data for Gandaki Province...
Fetching hourly data for Lumbini Province...


Fetching hourly data for Karnali Province...
Failed to fetch data for Karnali Province: 'Hourly' object has no attribute '_types'
Fetching hourly data for Sudurpashchim Province...
                     Division       time  temp_mean  temp_max  temp_min  \
16045  Sudurpashchim Province 2025-10-11       <NA>      <NA>      <NA>   
16046  Sudurpashchim Province 2025-10-12       <NA>      <NA>      <NA>   
16047  Sudurpashchim Province 2025-10-13       <NA>      <NA>      <NA>   
16048  Sudurpashchim Province 2025-10-14       <NA>      <NA>      <NA>   
16049  Sudurpashchim Province 2025-10-15       <NA>      <NA>      <NA>   

       rhum_mean  prcp_sum  wspd_mean  tsun_sum Country  \
16045       <NA>      <NA>       <NA>      <NA>   Nepal   
16046       <NA>      <NA>       <NA>      <NA>   Nepal   
16047       <NA>      <NA>       <NA>      <NA>   Nepal   
16048       <NA>      <NA>       <NA>      <NA>   Nepal   
16049       <NA>      <NA>       <NA>      <NA>   Nepal   

       temp_m

### Forecast

In [25]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data before forecasting
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]

forecast_list = []

# ✅ Loop through each Division and forecast all variables
for division in daily_df['Division'].unique():
    division_data = daily_df[daily_df['Division'] == division].copy()
    division_data.set_index('time', inplace=True)

    # Get the last confirmed date for this division
    last_date = division_data.index.max()

    # Temporary dict to store forecasts for this division
    division_forecasts = {'Division': [], 'time': []}

    for var in climate_vars:
        series = division_data[var].dropna()

        if len(series) > 30:
            try:
                # Fit SARIMAX model
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                # ✅ Forecast next 7 days
                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                # Initialize division & dates on first variable
                if var == climate_vars[0]:
                    division_forecasts['Division'] = [division] * 7
                    division_forecasts['time'] = future_dates

                # Add forecast and CI columns
                division_forecasts[var] = forecast.values
                division_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                division_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {division}: {e}")
                division_forecasts[var] = [None] * 7
                division_forecasts[f"{var}_lower95"] = [None] * 7
                division_forecasts[f"{var}_upper95"] = [None] * 7

    # Create DataFrame for this division forecast
    forecast_df = pd.DataFrame(division_forecasts)
    forecast_df['Country'] = 'Nepal'
    forecast_list.append(forecast_df)

# ✅ Combine forecasts for all divisions
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename columns to avoid overwriting historical values
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical daily_df
combined_df_with_forecast = pd.merge(
    daily_df,
    forecast_df_all,
    on=['Division', 'time', 'Country'],
    how='outer'
)

# ✅ Ensure datetime for merged DataFrame
combined_df_with_forecast['time'] = pd.to_datetime(combined_df_with_forecast['time'])

    # ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

 # ✅ Remove all *_provisional columns
combined_df_with_forecast = combined_df_with_forecast.drop(
        columns=[col for col in combined_df_with_forecast.columns if col.endswith('_provisional')],
        errors='ignore'
    )

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Division', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("nepal_states_weather_with_prediction_2022_2025.csv", index=False)

# Preview last rows
print(combined_df_with_forecast.tail(10))

                     Division       time  temp_mean  temp_max  temp_min  \
16025  Sudurpashchim Province 2025-10-03      26.35      31.0      21.4   
16026  Sudurpashchim Province 2025-10-04      26.39      31.3      22.2   
16027  Sudurpashchim Province 2025-10-05      25.95      30.5      21.5   
16028  Sudurpashchim Province 2025-10-06       <NA>      <NA>      <NA>   
16029  Sudurpashchim Province 2025-10-07       <NA>      <NA>      <NA>   
16030  Sudurpashchim Province 2025-10-08       <NA>      <NA>      <NA>   
16031  Sudurpashchim Province 2025-10-09       <NA>      <NA>      <NA>   
16032  Sudurpashchim Province 2025-10-10       <NA>      <NA>      <NA>   
16033  Sudurpashchim Province 2025-10-11       <NA>      <NA>      <NA>   
16034  Sudurpashchim Province 2025-10-12       <NA>      <NA>      <NA>   

       rhum_mean  prcp_sum  wspd_mean  tsun_sum Country  ...  \
16025      79.08       0.0       4.94       0.0   Nepal  ...   
16026       78.0       0.0        5.9       0.

### Forecast National Level

In [26]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]
daily_df = daily_df.drop(columns=['week_start'], errors='ignore')

# ✅ Aggregate to Country-level mean per day
country_df = (daily_df
              .groupby(['Country', 'time'])[climate_vars]
              .mean()
              .reset_index())

forecast_list = []

# ✅ Loop through each Country and forecast all variables
for country in country_df['Country'].unique():
    country_data = country_df[country_df['Country'] == country].copy()
    country_data.set_index('time', inplace=True)

    last_date = country_data.index.max()
    country_forecasts = {'Country': [], 'time': []}

    for var in climate_vars:
        series = country_data[var].dropna()

        if len(series) > 30:
            try:
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                if var == climate_vars[0]:
                    country_forecasts['Country'] = [country] * 7
                    country_forecasts['time'] = future_dates

                country_forecasts[var] = forecast.values
                country_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                country_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {country}: {e}")
                country_forecasts[var] = [None] * 7
                country_forecasts[f"{var}_lower95"] = [None] * 7
                country_forecasts[f"{var}_upper95"] = [None] * 7

    forecast_df = pd.DataFrame(country_forecasts)
    forecast_list.append(forecast_df)

# ✅ Combine all country forecasts
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename forecast columns
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical aggregated data
combined_df_with_forecast = pd.merge(
    country_df,
    forecast_df_all,
    on=['Country', 'time'],
    how='outer'
)

# ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Country', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("nepal_country_weather_with_prediction_2022_2025.csv", index=False)

# Preview
print(combined_df_with_forecast.tail(10))

     Country       time  temp_mean  rhum_mean  prcp_sum  wspd_mean  tsun_sum  \
3197   Nepal 2025-10-03      24.78       88.5      5.08       7.51       0.0   
3198   Nepal 2025-10-04      23.56      90.72     58.08       9.54       0.0   
3199   Nepal 2025-10-05      23.99      86.08     27.12       6.61       0.0   
3200   Nepal 2025-10-06       <NA>       <NA>      <NA>       <NA>      <NA>   
3201   Nepal 2025-10-07       <NA>       <NA>      <NA>       <NA>      <NA>   
3202   Nepal 2025-10-08       <NA>       <NA>      <NA>       <NA>      <NA>   
3203   Nepal 2025-10-09       <NA>       <NA>      <NA>       <NA>      <NA>   
3204   Nepal 2025-10-10       <NA>       <NA>      <NA>       <NA>      <NA>   
3205   Nepal 2025-10-11       <NA>       <NA>      <NA>       <NA>      <NA>   
3206   Nepal 2025-10-12       <NA>       <NA>      <NA>       <NA>      <NA>   

      temp_mean_forecast  temp_mean_forecast_lower95  \
3197                 NaN                         NaN   
3198   

## Maldives

### Daily Historical Data

In [27]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import numpy as np

# Define time period
start = datetime(2017, 1, 1)
end = datetime(2025, 12, 31, 23, 59)

# Define Maldives key atolls or capital islands with approximate coordinates and elevation
maldives_divisions = {
    'Malé': Point(4.1755, 73.5093, 2),              # Capital
    'Addu City (Seenu)': Point(-0.6305, 73.1581, 2), # Hithadhoo
    'Fuvahmulah (Gnaviyani)': Point(-0.2987, 73.4240, 2),
    'Kulhudhuffushi (Haa Dhaalu)': Point(6.6226, 73.0706, 2),
    'Thinadhoo (Gaafu Dhaalu)': Point(0.5333, 73.4333, 2),
    'Dhidhdhoo (Haa Alif)': Point(6.8874, 73.1140, 2),
    'Naifaru (Lhaviyani)': Point(5.4442, 73.3652, 2),
    'Eydhafushi (Baa)': Point(5.1036, 73.0706, 2),
    'Fonadhoo (Laamu)': Point(1.8342, 73.5023, 2),
    'Nilandhoo (Faafu)': Point(3.0681, 72.9256, 2),
    'Villingili (Gaafu Alif)': Point(0.7586, 73.4326, 2),
    'Veymandoo (Thaa)': Point(2.1871, 73.0958, 2),
    'Dhuvaafaru (Raa)': Point(5.5891, 73.3475, 2),
    'Maamigili (Alif Dhaal)': Point(3.4721, 72.8350, 2)
}

# Initialize dictionary to store DataFrames
all_data = {}

# ✅ Function to calculate Relative Humidity using Magnus formula
def calc_rh(temp, dewpt):
    temp = np.array(temp, dtype=float)
    dewpt = np.array(dewpt, dtype=float)
    mask = (~np.isnan(temp)) & (~np.isnan(dewpt))
    rh = np.full(temp.shape, np.nan)
    rh[mask] = 100 * np.exp((17.625 * dewpt[mask]) / (243.04 + dewpt[mask])) / \
                      np.exp((17.625 * temp[mask]) / (243.04 + temp[mask]))
    return rh

# ✅ Fetch hourly data for each division
for division, point in maldives_divisions.items():
    try:
        print(f"Fetching hourly data for {division}...")
        data = Hourly(point, start, end).fetch()

        # Skip if no data
        if data.empty:
            print(f"No data available for {division}")
            continue

        data['Division'] = division

        # Calculate RH if missing
        if 'rhum' not in data.columns and 'temp' in data.columns and 'dwpt' in data.columns:
            data['rhum'] = calc_rh(data['temp'].values, data['dwpt'].values)

        all_data[division] = data

    except Exception as e:
        print(f"Failed to fetch data for {division}: {e}")

# ✅ Combine all data into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data.values()).reset_index()
else:
    combined_df = pd.DataFrame()
    print("No data fetched for any division.")

# ✅ Ensure 'time' is datetime
if not combined_df.empty:
    combined_df['time'] = pd.to_datetime(combined_df['time'])

    # ✅ Group by Division and resample to daily
    daily_df = combined_df.groupby('Division').resample('D', on='time').agg({
        'temp': ['mean', 'max', 'min'],   # Avg, Max, Min temperature
        'rhum': 'mean',                   # Avg relative humidity
        'prcp': 'sum',                    # Total daily precipitation
        'wspd': 'mean',                   # Avg wind speed
        'tsun': 'sum'                     # Sum of sun exposure
    }).reset_index()

    # ✅ Flatten multi-level columns
    daily_df.columns = ['Division', 'time', 'temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    # ✅ Add 'Country' column
    daily_df['Country'] = 'Maldives'

     # ✅ Create provisional columns (copy today & future values)
    today = datetime.now().date()
    climate_vars = ['temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    for col in climate_vars:
        new_col = f"{col}_provisional"
        # Copy today's and future values into _provisional
        daily_df[new_col] = np.where(daily_df['time'].dt.date >= today, daily_df[col], np.nan)

        # Clear only future values from original columns, keep today intact
        daily_df.loc[daily_df['time'].dt.date > today, col] = np.nan

    # ✅ Round and save
    daily_df = daily_df.round(2)

    # ✅ Save to CSV
    daily_df.to_csv("maldives_states_weather_daily_2022_2025.csv", index=False)

    # ✅ Check last few rows
    print(daily_df.tail())
else:
    print("Daily DataFrame is empty.")

Fetching hourly data for Malé...


Fetching hourly data for Addu City (Seenu)...


Fetching hourly data for Fuvahmulah (Gnaviyani)...
Failed to fetch data for Fuvahmulah (Gnaviyani): 'Hourly' object has no attribute '_types'
Fetching hourly data for Kulhudhuffushi (Haa Dhaalu)...


Fetching hourly data for Thinadhoo (Gaafu Dhaalu)...
Failed to fetch data for Thinadhoo (Gaafu Dhaalu): 'Hourly' object has no attribute '_types'
Fetching hourly data for Dhidhdhoo (Haa Alif)...
Fetching hourly data for Naifaru (Lhaviyani)...
Failed to fetch data for Naifaru (Lhaviyani): 'Hourly' object has no attribute '_types'
Fetching hourly data for Eydhafushi (Baa)...
Failed to fetch data for Eydhafushi (Baa): 'Hourly' object has no attribute '_types'
Fetching hourly data for Fonadhoo (Laamu)...
Fetching hourly data for Nilandhoo (Faafu)...
Failed to fetch data for Nilandhoo (Faafu): 'Hourly' object has no attribute '_types'
Fetching hourly data for Villingili (Gaafu Alif)...
Failed to fetch data for Villingili (Gaafu Alif): 'Hourly' object has no attribute '_types'
Fetching hourly data for Veymandoo (Thaa)...
Failed to fetch data for Veymandoo (Thaa): 'Hourly' object has no attribute '_types'
Fetching hourly data for Dhuvaafaru (Raa)...
Failed to fetch data for Dhuvaafaru (Raa): 

### Forecast

In [28]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data before forecasting
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]

forecast_list = []

# ✅ Loop through each Division and forecast all variables
for division in daily_df['Division'].unique():
    division_data = daily_df[daily_df['Division'] == division].copy()
    division_data.set_index('time', inplace=True)

    # Get the last confirmed date for this division
    last_date = division_data.index.max()

    # Temporary dict to store forecasts for this division
    division_forecasts = {'Division': [], 'time': []}

    for var in climate_vars:
        series = division_data[var].dropna()

        if len(series) > 30:
            try:
                # Fit SARIMAX model
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                # ✅ Forecast next 7 days
                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                # Initialize division & dates on first variable
                if var == climate_vars[0]:
                    division_forecasts['Division'] = [division] * 7
                    division_forecasts['time'] = future_dates

                # Add forecast and CI columns
                division_forecasts[var] = forecast.values
                division_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                division_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {division}: {e}")
                division_forecasts[var] = [None] * 7
                division_forecasts[f"{var}_lower95"] = [None] * 7
                division_forecasts[f"{var}_upper95"] = [None] * 7

    # Create DataFrame for this division forecast
    forecast_df = pd.DataFrame(division_forecasts)
    forecast_df['Country'] = 'Maldives'
    forecast_list.append(forecast_df)

# ✅ Combine forecasts for all divisions
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename columns to avoid overwriting historical values
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical daily_df
combined_df_with_forecast = pd.merge(
    daily_df,
    forecast_df_all,
    on=['Division', 'time', 'Country'],
    how='outer'
)

# ✅ Ensure datetime for merged DataFrame
combined_df_with_forecast['time'] = pd.to_datetime(combined_df_with_forecast['time'])

    # ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

 # ✅ Remove all *_provisional columns
combined_df_with_forecast = combined_df_with_forecast.drop(
        columns=[col for col in combined_df_with_forecast.columns if col.endswith('_provisional')],
        errors='ignore'
    )

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Division', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("maldives_states_weather_with_prediction_2022_2025.csv", index=False)

# Preview last rows
print(combined_df_with_forecast.tail(10))

      Division       time  temp_mean  temp_max  temp_min  rhum_mean  prcp_sum  \
16025     Malé 2025-10-03      30.08      32.0      28.0      74.12       0.0   
16026     Malé 2025-10-04      30.35      33.0      29.0      76.29       0.0   
16027     Malé 2025-10-05      29.91      33.0      28.4      76.42       0.0   
16028     Malé 2025-10-06       <NA>      <NA>      <NA>       <NA>      <NA>   
16029     Malé 2025-10-07       <NA>      <NA>      <NA>       <NA>      <NA>   
16030     Malé 2025-10-08       <NA>      <NA>      <NA>       <NA>      <NA>   
16031     Malé 2025-10-09       <NA>      <NA>      <NA>       <NA>      <NA>   
16032     Malé 2025-10-10       <NA>      <NA>      <NA>       <NA>      <NA>   
16033     Malé 2025-10-11       <NA>      <NA>      <NA>       <NA>      <NA>   
16034     Malé 2025-10-12       <NA>      <NA>      <NA>       <NA>      <NA>   

       wspd_mean  tsun_sum   Country  ...  rhum_mean_forecast_upper95  \
16025       5.92       0.0  Maldive

### Forecast National Level

In [29]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]
daily_df = daily_df.drop(columns=['week_start'], errors='ignore')

# ✅ Aggregate to Country-level mean per day
country_df = (daily_df
              .groupby(['Country', 'time'])[climate_vars]
              .mean()
              .reset_index())

forecast_list = []

# ✅ Loop through each Country and forecast all variables
for country in country_df['Country'].unique():
    country_data = country_df[country_df['Country'] == country].copy()
    country_data.set_index('time', inplace=True)

    last_date = country_data.index.max()
    country_forecasts = {'Country': [], 'time': []}

    for var in climate_vars:
        series = country_data[var].dropna()

        if len(series) > 30:
            try:
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                if var == climate_vars[0]:
                    country_forecasts['Country'] = [country] * 7
                    country_forecasts['time'] = future_dates

                country_forecasts[var] = forecast.values
                country_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                country_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {country}: {e}")
                country_forecasts[var] = [None] * 7
                country_forecasts[f"{var}_lower95"] = [None] * 7
                country_forecasts[f"{var}_upper95"] = [None] * 7

    forecast_df = pd.DataFrame(country_forecasts)
    forecast_list.append(forecast_df)

# ✅ Combine all country forecasts
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename forecast columns
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical aggregated data
combined_df_with_forecast = pd.merge(
    country_df,
    forecast_df_all,
    on=['Country', 'time'],
    how='outer'
)

# ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Country', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("maldives_country_weather_with_prediction_2022_2025.csv", index=False)

# Preview
print(combined_df_with_forecast.tail(10))

       Country       time  temp_mean  rhum_mean  prcp_sum  wspd_mean  \
3197  Maldives 2025-10-03       29.1      75.19       0.2       8.51   
3198  Maldives 2025-10-04      29.16      75.67      2.28       9.16   
3199  Maldives 2025-10-05      28.83      74.52      0.82       11.7   
3200  Maldives 2025-10-06       <NA>       <NA>      <NA>       <NA>   
3201  Maldives 2025-10-07       <NA>       <NA>      <NA>       <NA>   
3202  Maldives 2025-10-08       <NA>       <NA>      <NA>       <NA>   
3203  Maldives 2025-10-09       <NA>       <NA>      <NA>       <NA>   
3204  Maldives 2025-10-10       <NA>       <NA>      <NA>       <NA>   
3205  Maldives 2025-10-11       <NA>       <NA>      <NA>       <NA>   
3206  Maldives 2025-10-12       <NA>       <NA>      <NA>       <NA>   

      tsun_sum  temp_mean_forecast  temp_mean_forecast_lower95  \
3197       0.0                 NaN                         NaN   
3198       0.0                 NaN                         NaN   
3199     

## Timor-Leste

### Daily Historical Data

In [30]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import numpy as np

# Define time period
start = datetime(2017, 1, 1)
end = datetime(2025, 12, 31, 23, 59)

# Define Timor-Leste municipalities with central coordinates and approximate elevation
timor_leste_municipalities = {
    'Aileu': Point(-8.7267, 125.5660, 900),         # Aileu
    'Ainaro': Point(-9.1653, 125.4930, 1080),       # Ainaro
    'Baucau': Point(-8.4711, 126.4560, 512),        # Baucau
    'Bobonaro': Point(-9.0084, 125.2153, 442),      # Maliana
    'Cova Lima': Point(-9.2970, 125.2534, 243),     # Suai
    'Dili': Point(-8.5586, 125.5736, 55),           # Dili (capital)
    'Ermera': Point(-8.7512, 125.3961, 1060),       # Gleno
    'Lautem': Point(-8.3852, 127.0335, 150),        # Lospalos
    'Liquiçá': Point(-8.5833, 125.3333, 250),       # Liquiçá
    'Manatuto': Point(-8.5167, 126.0167, 111),      # Manatuto
    'Manufahi': Point(-9.0083, 125.6533, 350),      # Same
    'Oecusse': Point(-9.2186, 124.3422, 27),        # Pante Macassar
    'Viqueque': Point(-8.8612, 126.3687, 120),      # Viqueque
}


# Initialize dictionary to store DataFrames
all_data = {}

# ✅ Function to calculate Relative Humidity using Magnus formula
def calc_rh(temp, dewpt):
    temp = np.array(temp, dtype=float)
    dewpt = np.array(dewpt, dtype=float)
    mask = (~np.isnan(temp)) & (~np.isnan(dewpt))
    rh = np.full(temp.shape, np.nan)
    rh[mask] = 100 * np.exp((17.625 * dewpt[mask]) / (243.04 + dewpt[mask])) / \
                      np.exp((17.625 * temp[mask]) / (243.04 + temp[mask]))
    return rh

# ✅ Fetch hourly data for each division
for division, point in timor_leste_municipalities.items():
    try:
        print(f"Fetching hourly data for {division}...")
        data = Hourly(point, start, end).fetch()

        # Skip if no data
        if data.empty:
            print(f"No data available for {division}")
            continue

        data['Division'] = division

        # Calculate RH if missing
        if 'rhum' not in data.columns and 'temp' in data.columns and 'dwpt' in data.columns:
            data['rhum'] = calc_rh(data['temp'].values, data['dwpt'].values)

        all_data[division] = data

    except Exception as e:
        print(f"Failed to fetch data for {division}: {e}")

# ✅ Combine all data into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data.values()).reset_index()
else:
    combined_df = pd.DataFrame()
    print("No data fetched for any division.")

# ✅ Ensure 'time' is datetime
if not combined_df.empty:
    combined_df['time'] = pd.to_datetime(combined_df['time'])

    # ✅ Group by Division and resample to daily
    daily_df = combined_df.groupby('Division').resample('D', on='time').agg({
        'temp': ['mean', 'max', 'min'],   # Avg, Max, Min temperature
        'rhum': 'mean',                   # Avg relative humidity
        'prcp': 'sum',                    # Total daily precipitation
        'wspd': 'mean',                   # Avg wind speed
        'tsun': 'sum'                     # Sum of sun exposure
    }).reset_index()

    # ✅ Flatten multi-level columns
    daily_df.columns = ['Division', 'time', 'temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    # ✅ Add 'Country' column
    daily_df['Country'] = 'Timor-Leste'

     # ✅ Create provisional columns (copy today & future values)
    today = datetime.now().date()
    climate_vars = ['temp_mean', 'temp_max', 'temp_min', 'rhum_mean', 'prcp_sum', 'wspd_mean', 'tsun_sum']

    for col in climate_vars:
        new_col = f"{col}_provisional"
        # Copy today's and future values into _provisional
        daily_df[new_col] = np.where(daily_df['time'].dt.date >= today, daily_df[col], np.nan)

        # Clear only future values from original columns, keep today intact
        daily_df.loc[daily_df['time'].dt.date > today, col] = np.nan

    # ✅ Round and save
    daily_df = daily_df.round(2)

    # ✅ Save to CSV
    daily_df.to_csv("timor_leste_states_weather_daily_2022_2025.csv", index=False)

    # ✅ Check last few rows
    print(daily_df.tail())
else:
    print("Daily DataFrame is empty.")

Fetching hourly data for Aileu...
Failed to fetch data for Aileu: 'Hourly' object has no attribute '_types'
Fetching hourly data for Ainaro...
Failed to fetch data for Ainaro: 'Hourly' object has no attribute '_types'
Fetching hourly data for Baucau...


Fetching hourly data for Bobonaro...
Failed to fetch data for Bobonaro: 'Hourly' object has no attribute '_types'
Fetching hourly data for Cova Lima...
Failed to fetch data for Cova Lima: 'Hourly' object has no attribute '_types'
Fetching hourly data for Dili...


Fetching hourly data for Ermera...
Failed to fetch data for Ermera: 'Hourly' object has no attribute '_types'
Fetching hourly data for Lautem...
Failed to fetch data for Lautem: 'Hourly' object has no attribute '_types'
Fetching hourly data for Liquiçá...
Fetching hourly data for Manatuto...
Failed to fetch data for Manatuto: 'Hourly' object has no attribute '_types'
Fetching hourly data for Manufahi...
Failed to fetch data for Manufahi: 'Hourly' object has no attribute '_types'
Fetching hourly data for Oecusse...


Fetching hourly data for Viqueque...
Failed to fetch data for Viqueque: 'Hourly' object has no attribute '_types'
     Division       time  temp_mean  temp_max  temp_min  rhum_mean  prcp_sum  \
4291  Oecusse 2025-10-10       <NA>      <NA>      <NA>       <NA>      <NA>   
4292  Oecusse 2025-10-11       <NA>      <NA>      <NA>       <NA>      <NA>   
4293  Oecusse 2025-10-12       <NA>      <NA>      <NA>       <NA>      <NA>   
4294  Oecusse 2025-10-13       <NA>      <NA>      <NA>       <NA>      <NA>   
4295  Oecusse 2025-10-14       <NA>      <NA>      <NA>       <NA>      <NA>   

      wspd_mean  tsun_sum      Country  temp_mean_provisional  \
4291       <NA>      <NA>  Timor-Leste                  28.62   
4292       <NA>      <NA>  Timor-Leste                  27.80   
4293       <NA>      <NA>  Timor-Leste                  28.02   
4294       <NA>      <NA>  Timor-Leste                  28.18   
4295       <NA>      <NA>  Timor-Leste                  29.20   

      temp_max

### Forecast

In [31]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data before forecasting
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]

forecast_list = []

# ✅ Loop through each Division and forecast all variables
for division in daily_df['Division'].unique():
    division_data = daily_df[daily_df['Division'] == division].copy()
    division_data.set_index('time', inplace=True)

    # Get the last confirmed date for this division
    last_date = division_data.index.max()

    # Temporary dict to store forecasts for this division
    division_forecasts = {'Division': [], 'time': []}

    for var in climate_vars:
        series = division_data[var].dropna()

        if len(series) > 30:
            try:
                # Fit SARIMAX model
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                # ✅ Forecast next 7 days
                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                # Initialize division & dates on first variable
                if var == climate_vars[0]:
                    division_forecasts['Division'] = [division] * 7
                    division_forecasts['time'] = future_dates

                # Add forecast and CI columns
                division_forecasts[var] = forecast.values
                division_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                division_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {division}: {e}")
                division_forecasts[var] = [None] * 7
                division_forecasts[f"{var}_lower95"] = [None] * 7
                division_forecasts[f"{var}_upper95"] = [None] * 7

    # Create DataFrame for this division forecast
    forecast_df = pd.DataFrame(division_forecasts)
    forecast_df['Country'] = 'Timor-Leste'
    forecast_list.append(forecast_df)

# ✅ Combine forecasts for all divisions
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename columns to avoid overwriting historical values
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical daily_df
combined_df_with_forecast = pd.merge(
    daily_df,
    forecast_df_all,
    on=['Division', 'time', 'Country'],
    how='outer'
)

# ✅ Ensure datetime for merged DataFrame
combined_df_with_forecast['time'] = pd.to_datetime(combined_df_with_forecast['time'])

    # ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

 # ✅ Remove all *_provisional columns
combined_df_with_forecast = combined_df_with_forecast.drop(
        columns=[col for col in combined_df_with_forecast.columns if col.endswith('_provisional')],
        errors='ignore'
    )

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Division', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("timor_leste_states_weather_with_prediction_2022_2025.csv", index=False)

# Preview last rows
print(combined_df_with_forecast.tail(10))

     Division       time  temp_mean  temp_max  temp_min  rhum_mean  prcp_sum  \
4296  Oecusse 2025-10-03      27.21      30.1      23.1      75.42       0.5   
4297  Oecusse 2025-10-04      27.74      31.3      23.5      69.83       0.0   
4298  Oecusse 2025-10-05      27.46      31.1      23.2      73.29       0.0   
4299  Oecusse 2025-10-06       <NA>      <NA>      <NA>       <NA>      <NA>   
4300  Oecusse 2025-10-07       <NA>      <NA>      <NA>       <NA>      <NA>   
4301  Oecusse 2025-10-08       <NA>      <NA>      <NA>       <NA>      <NA>   
4302  Oecusse 2025-10-09       <NA>      <NA>      <NA>       <NA>      <NA>   
4303  Oecusse 2025-10-10       <NA>      <NA>      <NA>       <NA>      <NA>   
4304  Oecusse 2025-10-11       <NA>      <NA>      <NA>       <NA>      <NA>   
4305  Oecusse 2025-10-12       <NA>      <NA>      <NA>       <NA>      <NA>   

      wspd_mean  tsun_sum      Country  ...  rhum_mean_forecast_upper95  \
4296      12.56       0.0  Timor-Leste  ... 

### Forecast National Level

In [32]:
from datetime import timedelta, datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd

# List of climate variables to forecast
climate_vars = ['temp_mean', 'rhum_mean', 'prcp_sum',
                'wspd_mean', 'tsun_sum']

# ✅ Remove provisional data
today = datetime.now().date()
daily_df = daily_df[daily_df['time'].dt.date <= today]
daily_df = daily_df.drop(columns=['week_start'], errors='ignore')

# ✅ Aggregate to Country-level mean per day
country_df = (daily_df
              .groupby(['Country', 'time'])[climate_vars]
              .mean()
              .reset_index())

forecast_list = []

# ✅ Loop through each Country and forecast all variables
for country in country_df['Country'].unique():
    country_data = country_df[country_df['Country'] == country].copy()
    country_data.set_index('time', inplace=True)

    last_date = country_data.index.max()
    country_forecasts = {'Country': [], 'time': []}

    for var in climate_vars:
        series = country_data[var].dropna()

        if len(series) > 30:
            try:
                model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False, enforce_invertibility=False)
                results = model.fit(disp=False)

                future_dates = pd.date_range(last_date + timedelta(days=1), periods=7, freq='D')
                pred = results.get_forecast(steps=7)
                forecast = pred.predicted_mean.round(2)
                conf_int = pred.conf_int(alpha=0.05).round(2)

                if var == climate_vars[0]:
                    country_forecasts['Country'] = [country] * 7
                    country_forecasts['time'] = future_dates

                country_forecasts[var] = forecast.values
                country_forecasts[f"{var}_lower95"] = conf_int.iloc[:, 0].values
                country_forecasts[f"{var}_upper95"] = conf_int.iloc[:, 1].values

            except Exception as e:
                print(f"Failed to forecast {var} for {country}: {e}")
                country_forecasts[var] = [None] * 7
                country_forecasts[f"{var}_lower95"] = [None] * 7
                country_forecasts[f"{var}_upper95"] = [None] * 7

    forecast_df = pd.DataFrame(country_forecasts)
    forecast_list.append(forecast_df)

# ✅ Combine all country forecasts
forecast_df_all = pd.concat(forecast_list, ignore_index=True)

# ✅ Rename forecast columns
for col in climate_vars:
    forecast_df_all.rename(columns={
        col: f"{col}_forecast",
        f"{col}_lower95": f"{col}_forecast_lower95",
        f"{col}_upper95": f"{col}_forecast_upper95"
    }, inplace=True)

# ✅ Merge forecasts into historical aggregated data
combined_df_with_forecast = pd.merge(
    country_df,
    forecast_df_all,
    on=['Country', 'time'],
    how='outer'
)

# ✅ Copy today's actual values into _forecast columns
for col in climate_vars:
        forecast_col = f"{col}_forecast"
        combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, forecast_col
        ] = combined_df_with_forecast.loc[
            combined_df_with_forecast['time'].dt.date == today, col
        ]

# Sort and round
combined_df_with_forecast = combined_df_with_forecast.sort_values(by=['Country', 'time']).reset_index(drop=True)
combined_df_with_forecast = combined_df_with_forecast.round(2)

# ✅ Save to CSV
combined_df_with_forecast.to_csv("timor_leste_country_weather_with_prediction_2022_2025.csv", index=False)

# Preview
print(combined_df_with_forecast.tail(10))

          Country       time  temp_mean  rhum_mean  prcp_sum  wspd_mean  \
2136  Timor-Leste 2025-10-03       25.8      70.67      0.25       11.8   
2137  Timor-Leste 2025-10-04      26.34       67.4       0.0      13.38   
2138  Timor-Leste 2025-10-05      25.81      72.52       0.0      13.46   
2139  Timor-Leste 2025-10-06       <NA>       <NA>      <NA>       <NA>   
2140  Timor-Leste 2025-10-07       <NA>       <NA>      <NA>       <NA>   
2141  Timor-Leste 2025-10-08       <NA>       <NA>      <NA>       <NA>   
2142  Timor-Leste 2025-10-09       <NA>       <NA>      <NA>       <NA>   
2143  Timor-Leste 2025-10-10       <NA>       <NA>      <NA>       <NA>   
2144  Timor-Leste 2025-10-11       <NA>       <NA>      <NA>       <NA>   
2145  Timor-Leste 2025-10-12       <NA>       <NA>      <NA>       <NA>   

      tsun_sum  temp_mean_forecast  temp_mean_forecast_lower95  \
2136       0.0                 NaN                         NaN   
2137       0.0                 NaN        

## Combine

### Forecast state

In [33]:
import pandas as pd
import glob
import os

# ✅ Set working directory to source folder
source_dir = r'C:\Users\dhihr\OneDrive\bahan who\kerjaan who\climate'
os.chdir(source_dir)

# ✅ Pattern to match files
pattern = os.path.join(source_dir, "*states_weather_with_prediction_2022_2025.csv")

# ✅ Get list of matching files
files = glob.glob(pattern)
print(f"Found {len(files)} files to combine.")

# ✅ Read and combine
dfs = [pd.read_csv(f) for f in files]
combined_df = pd.concat(dfs, ignore_index=True)

# ✅ Save combined file to both destinations
output_filename = "combined_states_weather_with_prediction_2022_2025.csv"

# Save to OneDrive folder
combined_df.to_csv(r'C:\Users\dhihr\OneDrive\bahan who\kerjaan who\climate\\' + output_filename, index=False)

# Save to GitHub folder
source_dir2 = r'C:\Users\dhihr\Documents\GitHub\WHO_SEAR\climate'
combined_df.to_csv(os.path.join(source_dir2, output_filename), index=False)

print(f"✅ Combined CSV saved as {output_filename} in both locations.")

Found 9 files to combine.
✅ Combined CSV saved as combined_states_weather_with_prediction_2022_2025.csv in both locations.


### Forecast National Level

In [34]:
import pandas as pd
import glob
import os

# ✅ Set working directory to source folder
source_dir = r'C:\Users\dhihr\OneDrive\bahan who\kerjaan who\climate'
os.chdir(source_dir)

# ✅ Pattern to match files
pattern = os.path.join(os.getcwd(), "*country_weather_with_prediction_2022_2025.csv")

# ✅ Get list of matching files
files = glob.glob(pattern)
print(f"Found {len(files)} files to combine.")

# ✅ Read and combine
dfs = [pd.read_csv(f) for f in files]
combined_df = pd.concat(dfs, ignore_index=True)

# ✅ Save combined file
# Save to OneDrive folder
os.chdir(r'C:\Users\dhihr\OneDrive\bahan who\kerjaan who\climate')
combined_df.to_csv("combined_country_weather_with_prediction_2022_2025.csv", index=False)

# Save to GitHub folder
source_dir2 = r'C:\Users\dhihr\Documents\GitHub\WHO_SEAR\climate'
os.chdir(source_dir2)
combined_df.to_csv("combined_country_weather_with_prediction_2022_2025.csv", index=False)

print("✅ Combined CSV saved as combined_country_weather_with_prediction_2022_2025.csv")

Found 9 files to combine.
✅ Combined CSV saved as combined_country_weather_with_prediction_2022_2025.csv
